In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import scipy.sparse as sps

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

dataset_dir = "../input/dataset/"

# Interactions files (URM)
data_train = dataset_dir + "data_train.csv"
data_target_users = dataset_dir + "data_target_users_test.csv"

# Item content files (ICM)
data_ICM_asset = dataset_dir + "/data_ICM_asset.csv"  # description of the item (id)
data_ICM_price = dataset_dir + "/data_ICM_price.csv"  # price of each item (already normalized)
data_ICM_sub_class = dataset_dir + "/data_ICM_sub_class.csv"  # categorization of the item (number)

# Any results you write to the current directory are saved as output.


/kaggle/input/dataset/data_UCM_age.csv
/kaggle/input/dataset/data_ICM_sub_class.csv
/kaggle/input/dataset/data_ICM_asset.csv
/kaggle/input/dataset/data_ICM_price.csv
/kaggle/input/dataset/data_UCM_region.csv
/kaggle/input/dataset/data_target_users_test.csv
/kaggle/input/dataset/data_train.csv


> ** Load data and preprocess**

In [2]:
# data_manager.py
# ------------------------------------------------------------------

# global vars
user_list = []
item_list = []
n_interactions = 0
n_users = 0
n_items = 0
n_subclass = 0


def build_URM():
    global user_list, item_list, n_interactions

    matrix_tuples = []

    with open(data_train, 'r') as file:  # read file's content
        next(file)  # skip header row
        for line in file:
            if len(line.strip()) != 0:  # ignore lines with only whitespace
                n_interactions += 1

                # Create a tuple for each interaction (line in the file)
                matrix_tuples.append(row_split(line))

    # Separate user_id, item_id and rating
    user_list, item_list, rating_list = zip(*matrix_tuples)  # join tuples together (zip() to map values)

    # Create lists of all users, items and contents (ratings)
    user_list = list(user_list)  # row
    item_list = list(item_list)  # col
    rating_list = list(rating_list)  # data

    URM = csr_sparse_matrix(rating_list, user_list, item_list)
    
    print("URM built!")

    return URM

def build_ICM():
    # features = [‘asset’, ’price’, ’subclass’] info about products
    global n_subclass

    # Load subclass data
    matrix_tuples = []

    with open(data_ICM_sub_class, 'r') as file:  # read file's content
        next(file)  # skip header row
        for line in file:
            n_subclass += 1

            # Create a tuple for each interaction (line in the file)
            matrix_tuples.append(row_split(line))

    # Separate user_id, item_id and rating
    item_list, class_list, col_list = zip(*matrix_tuples)  # join tuples together (zip() to map values)

    # Convert values to list# Create lists of all users, items and contents (ratings)
    item_list_icm = list(item_list)
    class_list_icm = list(class_list)
    col_list_icm = np.zeros(len(col_list))

    # Number of items that are in the subclass list
    num_items = max(item_list_icm) + 1
    ICM_shape = (num_items, 1)
    ICM_subclass = csr_sparse_matrix(class_list_icm, item_list_icm, col_list_icm, shape=ICM_shape)

    # Load price data
    matrix_tuples = []
    n_prices = 0

    with open(data_ICM_price, 'r') as file:  # read file's content
        next(file)  # skip header row
        for line in file:
            n_prices += 1

            # Create a tuple for each interaction (line in the file)
            matrix_tuples.append(row_split(line))

    # Separate user_id, item_id and rating
    item_list, col_list, price_list = zip(*matrix_tuples)  # join tuples together (zip() to map values)

    # Convert values to list# Create lists of all users, items and contents (ratings)
    item_list_icm = list(item_list)
    col_list_icm = list(col_list)
    price_list_icm = list(price_list)

    ICM_price = csr_sparse_matrix(price_list_icm, item_list_icm, col_list_icm)

    # Load asset data
    matrix_tuples = []
    n_assets = 0

    with open(data_ICM_asset, 'r') as file:  # read file's content
        next(file)  # skip header row
        for line in file:
            n_assets += 1

            # Create a tuple for each interaction (line in the file)
            matrix_tuples.append(row_split(line))

    # Separate user_id, item_id and rating
    item_list, col_list, asset_list = zip(*matrix_tuples)  # join tuples together (zip() to map values)

    # Convert values to list# Create lists of all users, items and contents (ratings)
    item_list_icm = list(item_list)
    col_list_icm = list(col_list)
    asset_list_icm = list(asset_list)

    ICM_asset = csr_sparse_matrix(asset_list_icm, item_list_icm, col_list_icm)

    ICM_all = sps.hstack([ICM_price, ICM_asset, ICM_subclass], format='csr')

    # item_feature_ratios(ICM_all)

    print("ICM built!\n")

    return ICM_all

def row_split(row_string):
    # file format: 0,3568,1.0

    split = row_string.split(",")
    split[2] = split[2].replace("\n", "")

    split[0] = int(split[0])
    split[1] = int(split[1])
    split[2] = float(split[2])  # rating is a float

    result = tuple(split)
    return result

def csr_sparse_matrix(data, row, col, shape=None):
    csr_matrix = sps.coo_matrix((data, (row, col)), shape=shape)
    csr_matrix = csr_matrix.tocsr()

    return csr_matrix


def get_target_users():
    target_user_id_list = []

    with open(data_target_users, 'r') as file:  # read file's content
        next(file)  # skip header row
        for line in file:
            # each line is a user_id
            target_user_id_list.append(int(line.strip()))  # remove trailing space

    return target_user_id_list

> **Train, validation and test splitting**

In [3]:
# data_splitter.py
# ------------------------------------------------------------------

# Random holdout split: take interactions randomly
# and do not care about which users were involved in that interaction

def split_train_validation_random_holdout(URM, train_split):
    number_interactions = URM.nnz  # number of nonzero values
    URM = URM.tocoo()  # Coordinate list matrix (COO)
    shape = URM.shape

    #  URM.row: user_list, URM.col: item_list, URM.data: rating_list

    # Sampling strategy: take random samples of data using a boolean mask
    train_mask = np.random.choice(
        [True, False],
        number_interactions,
        p=[train_split, 1 - train_split])  # train_perc for True, 1-train_perc for False

    URM_train = csr_sparse_matrix(URM.data[train_mask],
                                  URM.row[train_mask],
                                  URM.col[train_mask],
                                  shape=shape)

    test_mask = np.logical_not(train_mask)  # remaining samples
    URM_test = csr_sparse_matrix(URM.data[test_mask],
                                 URM.row[test_mask],
                                 URM.col[test_mask],
                                 shape=shape)

    return URM_train, URM_test



> **Performance evaluation**

In [4]:
# Incremental_Training_Early_Stopping.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 06/07/2018
@author: Maurizio Ferrari Dacrema
"""

import time, sys
import numpy as np
# from Base.BaseTempFolder import BaseTempFolder
# from utils.Evaluation.Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


class Incremental_Training_Early_Stopping(object):
    """
    This class provides a function which trains a model applying early stopping
    The term "incremental" refers to the model that is updated at every epoch
    The term "best" refers to the incremental model which corresponded to the best validation score
    The object must implement the following methods:
    _run_epoch(self, num_epoch)                 : trains the model for one epoch (e.g. calling another object implementing the training cython, pyTorch...)
    _prepare_model_for_validation(self)         : ensures the recommender being trained can compute the predictions needed for the validation step
    _update_best_model(self)                    : updates the best model with the current incremental one
    _train_with_early_stopping(.)               : Function that executes the training, validation and early stopping by using the previously implemented functions
    """

    def __init__(self):
        super(Incremental_Training_Early_Stopping, self).__init__()


    def get_early_stopping_final_epochs_dict(self):
        """
        This function returns a dictionary to be used as optimal parameters in the .fit() function
        It provides the flexibility to deal with multiple early-stopping in a single algorithm
        e.g. in NeuMF there are three model components each with its own optimal number of epochs
        the return dict would be {"epochs": epochs_best_neumf, "epochs_gmf": epochs_best_gmf, "epochs_mlp": epochs_best_mlp}
        :return:
        """

        return {"epochs": self.epochs_best}


    def _run_epoch(self, num_epoch):
        """
        This function should run a single epoch on the object you train. This may either involve calling a function to do an epoch
        on a Cython object or a loop on the data points directly in python
        :param num_epoch:
        :return:
        """
        raise NotImplementedError()


    def _prepare_model_for_validation(self):
        """
        This function is executed before the evaluation of the current model
        It should ensure the current object "self" can be passed to the evaluator object
        E.G. if the epoch is done via Cython or PyTorch, this function should get the new parameter values from
        the cython or pytorch objects into the self. pyhon object
        :return:
        """
        raise NotImplementedError()


    def _update_best_model(self):
        """
        This function is called when the incremental model is found to have better validation score than the current best one
        So the current best model should be replaced by the current incremental one.
        Important, remember to clone the objects and NOT to create a pointer-reference, otherwise the best solution will be altered
        by the next epoch
        :return:
        """
        raise NotImplementedError()



    def _train_with_early_stopping(self, epochs_max, epochs_min = 0,
                                   validation_every_n = None, stop_on_validation = False,
                                   validation_metric = None, lower_validations_allowed = None, evaluator_object = None,
                                   algorithm_name = "Incremental_Training_Early_Stopping"):
        """
        :param epochs_max:                  max number of epochs the training will last
        :param epochs_min:                  min number of epochs the training will last
        :param validation_every_n:          number of epochs after which the model will be evaluated and a best_model selected
        :param stop_on_validation:          [True/False] whether to stop the training before the max number of epochs
        :param validation_metric:           which metric to use when selecting the best model, higher values are better
        :param lower_validations_allowed:    number of contiguous validation steps required for the tranining to early-stop
        :param evaluator_object:            evaluator instance used to compute the validation metrics.
                                                If multiple cutoffs are available, the first one is used
        :param algorithm_name:              name of the algorithm to be displayed in the output updates
        :return: -
        Supported uses:
        - Train for max number of epochs with no validation nor early stopping:
            _train_with_early_stopping(epochs_max = 100,
                                        evaluator_object = None
                                        epochs_min,                 not used
                                        validation_every_n,         not used
                                        stop_on_validation,         not used
                                        validation_metric,          not used
                                        lower_validations_allowed,   not used
                                        )
        - Train for max number of epochs with validation but NOT early stopping:
            _train_with_early_stopping(epochs_max = 100,
                                        evaluator_object = evaluator
                                        stop_on_validation = False
                                        validation_every_n = int value
                                        validation_metric = metric name string
                                        epochs_min,                 not used
                                        lower_validations_allowed,   not used
                                        )
        - Train for max number of epochs with validation AND early stopping:
            _train_with_early_stopping(epochs_max = 100,
                                        epochs_min = int value
                                        evaluator_object = evaluator
                                        stop_on_validation = True
                                        validation_every_n = int value
                                        validation_metric = metric name string
                                        lower_validations_allowed = int value
                                        )
        """

        assert epochs_max >= 0, "{}: Number of epochs_max must be >= 0, passed was {}".format(algorithm_name, epochs_max)
        assert epochs_min >= 0, "{}: Number of epochs_min must be >= 0, passed was {}".format(algorithm_name, epochs_min)
        assert epochs_min <= epochs_max, "{}: epochs_min must be <= epochs_max, passed are epochs_min {}, epochs_max {}".format(algorithm_name, epochs_min, epochs_max)

        # Train for max number of epochs with no validation nor early stopping
        # OR Train for max number of epochs with validation but NOT early stopping
        # OR Train for max number of epochs with validation AND early stopping
        assert evaluator_object is None or\
               (evaluator_object is not None and not stop_on_validation and validation_every_n is not None and validation_metric is not None) or\
               (evaluator_object is not None and stop_on_validation and validation_every_n is not None and validation_metric is not None and lower_validations_allowed is not None),\
            "{}: Inconsistent parameters passed, please check the supported uses".format(algorithm_name)




        start_time = time.time()

        self.best_validation_metric = None
        lower_validatons_count = 0
        convergence = False

        self.epochs_best = 0

        epochs_current = 0

        while epochs_current < epochs_max and not convergence:

            self._run_epoch(epochs_current)

            # If no validation required, always keep the latest
            if evaluator_object is None:

                self.epochs_best = epochs_current

            # Determine whether a validaton step is required
            elif (epochs_current + 1) % validation_every_n == 0:

                print("{}: Validation begins...".format(algorithm_name))

                self._prepare_model_for_validation()

                # If the evaluator validation has multiple cutoffs, choose the first one
                results_run, results_run_string = evaluator_object.evaluateRecommender(self)
                results_run = results_run[list(results_run.keys())[0]]

                print("{}: {}".format(algorithm_name, results_run_string))

                # Update optimal model
                current_metric_value = results_run[validation_metric]
                #
                # if not np.isfinite(current_metric_value):
                #     if isinstance(self, BaseTempFolder):
                #         # If the recommender uses BaseTempFolder, clean the temp folder
                #         self._clean_temp_folder(temp_file_folder=self.temp_file_folder)
                #
                #     assert False, "{}: metric value is not a finite number, terminating!".format(self.RECOMMENDER_NAME)
                #

                if self.best_validation_metric is None or self.best_validation_metric < current_metric_value:

                    print("{}: New best model found! Updating.".format(algorithm_name))

                    self.best_validation_metric = current_metric_value

                    self._update_best_model()

                    self.epochs_best = epochs_current +1
                    lower_validatons_count = 0

                else:
                    lower_validatons_count += 1


                if stop_on_validation and lower_validatons_count >= lower_validations_allowed and epochs_current >= epochs_min:
                    convergence = True

                    elapsed_time = time.time() - start_time
                    new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)

                    print("{}: Convergence reached! Terminating at epoch {}. Best value for '{}' at epoch {} is {:.4f}. Elapsed time {:.2f} {}".format(
                        algorithm_name, epochs_current+1, validation_metric, self.epochs_best, self.best_validation_metric, new_time_value, new_time_unit))


            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)

            print("{}: Epoch {} of {}. Elapsed time {:.2f} {}".format(
                algorithm_name, epochs_current+1, epochs_max, new_time_value, new_time_unit))

            epochs_current += 1

            sys.stdout.flush()
            sys.stderr.flush()

        # If no validation required, keep the latest
        if evaluator_object is None:

            self._prepare_model_for_validation()
            self._update_best_model()


        # Stop when max epochs reached and not early-stopping
        if not convergence:
            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)

            if evaluator_object is not None and self.best_validation_metric is not None:
                print("{}: Terminating at epoch {}. Best value for '{}' at epoch {} is {:.4f}. Elapsed time {:.2f} {}".format(
                    algorithm_name, epochs_current, validation_metric, self.epochs_best, self.best_validation_metric, new_time_value, new_time_unit))
            else:
                print("{}: Terminating at epoch {}. Elapsed time {:.2f} {}".format(
                    algorithm_name, epochs_current, new_time_value, new_time_unit))

                
        

In [5]:
# Evaluator.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 26/06/18

@author: Maurizio Ferrari Dacrema
"""

import numpy as np
import scipy.sparse as sps
import time, sys, copy

from enum import Enum
# from utils.Evaluation.Utils.seconds_to_biggest_unit import seconds_to_biggest_unit

# from utils.Evaluation.metrics import roc_auc, precision, precision_recall_min_denominator, recall, MAP, MRR, ndcg, arhr, \
#     rmse, \
#     Novelty, Coverage_Item, Metrics_Object, Coverage_User, Gini_Diversity, Shannon_Entropy, Diversity_MeanInterList, \
#     Diversity_Herfindahl, AveragePopularity


class EvaluatorMetrics(Enum):
    ROC_AUC = "ROC_AUC"
    PRECISION = "PRECISION"
    PRECISION_RECALL_MIN_DEN = "PRECISION_RECALL_MIN_DEN"
    RECALL = "RECALL"
    MAP = "MAP"
    MRR = "MRR"
    NDCG = "NDCG"
    F1 = "F1"
    HIT_RATE = "HIT_RATE"
    ARHR = "ARHR"
    RMSE = "RMSE"
    NOVELTY = "NOVELTY"
    AVERAGE_POPULARITY = "AVERAGE_POPULARITY"
    DIVERSITY_SIMILARITY = "DIVERSITY_SIMILARITY"
    DIVERSITY_MEAN_INTER_LIST = "DIVERSITY_MEAN_INTER_LIST"
    DIVERSITY_HERFINDAHL = "DIVERSITY_HERFINDAHL"
    COVERAGE_ITEM = "COVERAGE_ITEM"
    COVERAGE_USER = "COVERAGE_USER"
    DIVERSITY_GINI = "DIVERSITY_GINI"
    SHANNON_ENTROPY = "SHANNON_ENTROPY"


def create_empty_metrics_dict(n_items, n_users, URM_train, ignore_items, ignore_users, cutoff,
                              diversity_similarity_object):
    empty_dict = {}

    # from Base.Evaluation.ResultMetric import ResultMetric
    # empty_dict = ResultMetric()

    for metric in EvaluatorMetrics:
        if metric == EvaluatorMetrics.COVERAGE_ITEM:
            empty_dict[metric.value] = Coverage_Item(n_items, ignore_items)

        elif metric == EvaluatorMetrics.DIVERSITY_GINI:
            empty_dict[metric.value] = Gini_Diversity(n_items, ignore_items)

        elif metric == EvaluatorMetrics.SHANNON_ENTROPY:
            empty_dict[metric.value] = Shannon_Entropy(n_items, ignore_items)

        elif metric == EvaluatorMetrics.COVERAGE_USER:
            empty_dict[metric.value] = Coverage_User(n_users, ignore_users)

        elif metric == EvaluatorMetrics.DIVERSITY_MEAN_INTER_LIST:
            empty_dict[metric.value] = Diversity_MeanInterList(n_items, cutoff)

        elif metric == EvaluatorMetrics.DIVERSITY_HERFINDAHL:
            empty_dict[metric.value] = Diversity_Herfindahl(n_items, ignore_items)

        elif metric == EvaluatorMetrics.NOVELTY:
            empty_dict[metric.value] = Novelty(URM_train)

        elif metric == EvaluatorMetrics.AVERAGE_POPULARITY:
            empty_dict[metric.value] = AveragePopularity(URM_train)

        elif metric == EvaluatorMetrics.MAP:
            empty_dict[metric.value] = MAP()

        elif metric == EvaluatorMetrics.MRR:
            empty_dict[metric.value] = MRR()

        elif metric == EvaluatorMetrics.DIVERSITY_SIMILARITY:
            if diversity_similarity_object is not None:
                empty_dict[metric.value] = copy.deepcopy(diversity_similarity_object)
        else:
            empty_dict[metric.value] = 0.0

    return empty_dict


def get_result_string(results_run, n_decimals=7):
    output_str = ""

    for cutoff in results_run.keys():

        results_run_current_cutoff = results_run[cutoff]

        output_str += "CUTOFF: {} - ".format(cutoff)

        for metric in results_run_current_cutoff.keys():
            output_str += "{}: {:.{n_decimals}f}, ".format(metric, results_run_current_cutoff[metric],
                                                           n_decimals=n_decimals)
        output_str += "\n"

    return output_str


def _remove_item_interactions(URM, item_list):
    URM = sps.csc_matrix(URM.copy())

    for item_index in item_list:
        start_pos = URM.indptr[item_index]
        end_pos = URM.indptr[item_index + 1]

        URM.data[start_pos:end_pos] = np.zeros_like(URM.data[start_pos:end_pos])

    URM.eliminate_zeros()
    URM = sps.csr_matrix(URM)

    return URM


class Evaluator(object):
    """Abstract Evaluator"""

    EVALUATOR_NAME = "Evaluator_Base_Class"

    def __init__(self, URM_test_list, cutoff_list, minRatingsPerUser=1, exclude_seen=True,
                 diversity_object=None,
                 ignore_items=None,
                 ignore_users=None,
                 verbose=True):

        super(Evaluator, self).__init__()

        self.verbose = verbose

        if ignore_items is None:
            self.ignore_items_flag = False
            self.ignore_items_ID = np.array([])
        else:
            self._print("Ignoring {} Items".format(len(ignore_items)))
            self.ignore_items_flag = True
            self.ignore_items_ID = np.array(ignore_items)

        self.cutoff_list = cutoff_list.copy()
        self.max_cutoff = max(self.cutoff_list)

        self.minRatingsPerUser = minRatingsPerUser
        self.exclude_seen = exclude_seen

        if not isinstance(URM_test_list, list):
            self.URM_test = URM_test_list.copy()
            URM_test_list = [URM_test_list]
        else:
            raise ValueError("List of URM_test not supported")

        self.diversity_object = diversity_object

        self.n_users, self.n_items = URM_test_list[0].shape

        # Prune users with an insufficient number of ratings
        # During testing CSR is faster
        self.URM_test_list = []
        usersToEvaluate_mask = np.zeros(self.n_users, dtype=np.bool)

        for URM_test in URM_test_list:
            URM_test = _remove_item_interactions(URM_test, self.ignore_items_ID)

            URM_test = sps.csr_matrix(URM_test)
            self.URM_test_list.append(URM_test)

            rows = URM_test.indptr
            numRatings = np.ediff1d(rows)
            new_mask = numRatings >= minRatingsPerUser

            usersToEvaluate_mask = np.logical_or(usersToEvaluate_mask, new_mask)

        self.usersToEvaluate = np.arange(self.n_users)[usersToEvaluate_mask]

        if ignore_users is not None:
            self._print("Ignoring {} Users".format(len(ignore_users)))
            self.ignore_users_ID = np.array(ignore_users)
            self.usersToEvaluate = set(self.usersToEvaluate) - set(ignore_users)
        else:
            self.ignore_users_ID = np.array([])

        self.usersToEvaluate = list(self.usersToEvaluate)

    def _print(self, string):

        if self.verbose:
            print("{}: {}".format(self.EVALUATOR_NAME, string))

    def evaluateRecommender(self, recommender_object):
        """
        :param recommender_object: the trained recommender object, a BaseRecommender subclass
        :param URM_test_list: list of URMs to test the recommender against, or a single URM object
        :param cutoff_list: list of cutoffs to be use to report the scores, or a single cutoff
        """

        raise NotImplementedError("The method evaluateRecommender not implemented for this evaluator class")

    def get_user_relevant_items(self, user_id):

        assert self.URM_test.getformat() == "csr", "Evaluator_Base_Class: URM_test is not CSR, this will cause errors in getting relevant items"

        return self.URM_test.indices[self.URM_test.indptr[user_id]:self.URM_test.indptr[user_id + 1]]

    def get_user_test_ratings(self, user_id):

        assert self.URM_test.getformat() == "csr", "Evaluator_Base_Class: URM_test is not CSR, this will cause errors in relevant items ratings"

        return self.URM_test.data[self.URM_test.indptr[user_id]:self.URM_test.indptr[user_id + 1]]


class EvaluatorHoldout(Evaluator):
    """EvaluatorHoldout"""

    EVALUATOR_NAME = "EvaluatorHoldout"

    def __init__(self, URM_test_list, cutoff_list, minRatingsPerUser=1, exclude_seen=True,
                 diversity_object=None,
                 ignore_items=None,
                 ignore_users=None,
                 verbose=True):

        super(EvaluatorHoldout, self).__init__(URM_test_list, cutoff_list,
                                               diversity_object=diversity_object,
                                               minRatingsPerUser=minRatingsPerUser, exclude_seen=exclude_seen,
                                               ignore_items=ignore_items, ignore_users=ignore_users,
                                               verbose=verbose)

    def _run_evaluation_on_selected_users(self, recommender_object, usersToEvaluate, block_size=None):

        if block_size is None:
            block_size = min(1000, int(1e8 / self.n_items))
            block_size = min(block_size, len(usersToEvaluate))

        start_time = time.time()
        start_time_print = time.time()

        results_dict = {}

        for cutoff in self.cutoff_list:
            results_dict[cutoff] = create_empty_metrics_dict(self.n_items, self.n_users,
                                                             recommender_object.get_URM_train(),
                                                             self.ignore_items_ID,
                                                             self.ignore_users_ID,
                                                             cutoff,
                                                             self.diversity_object)

        if self.ignore_items_flag:
            recommender_object.set_items_to_ignore(self.ignore_items_ID)

        n_users_evaluated = 0

        # Start from -block_size to ensure it to be 0 at the first block
        user_batch_start = 0
        user_batch_end = 0

        while user_batch_start < len(usersToEvaluate):

            user_batch_end = user_batch_start + block_size
            user_batch_end = min(user_batch_end, len(usersToEvaluate))

            test_user_batch_array = np.array(usersToEvaluate[user_batch_start:user_batch_end])
            user_batch_start = user_batch_end

            # Compute predictions for a batch of users using vectorization, much more efficient than computing it one at a time
            recommended_items_batch_list, scores_batch = recommender_object.recommend(test_user_batch_array,
                                                                                      remove_seen_flag=self.exclude_seen,
                                                                                      cutoff=self.max_cutoff,
                                                                                      remove_top_pop_flag=False,
                                                                                      remove_custom_items_flag=self.ignore_items_flag,
                                                                                      return_scores=True
                                                                                      )

            assert len(recommended_items_batch_list) == len(
                test_user_batch_array), "{}: recommended_items_batch_list contained recommendations for {} users, expected was {}".format(
                self.EVALUATOR_NAME, len(recommended_items_batch_list), len(test_user_batch_array))

            assert scores_batch.shape[0] == len(
                test_user_batch_array), "{}: scores_batch contained scores for {} users, expected was {}".format(
                self.EVALUATOR_NAME, scores_batch.shape[0], len(test_user_batch_array))

            assert scores_batch.shape[
                       1] == self.n_items, "{}: scores_batch contained scores for {} items, expected was {}".format(
                self.EVALUATOR_NAME, scores_batch.shape[1], self.n_items)

            # Compute recommendation quality for each user in batch
            for batch_user_index in range(len(recommended_items_batch_list)):

                test_user = test_user_batch_array[batch_user_index]

                relevant_items = self.get_user_relevant_items(test_user)
                relevant_items_rating = self.get_user_test_ratings(test_user)

                all_items_predicted_ratings = scores_batch[batch_user_index]
                user_rmse = rmse(all_items_predicted_ratings, relevant_items, relevant_items_rating)

                # Being the URM CSR, the indices are the non-zero column indexes
                recommended_items = recommended_items_batch_list[batch_user_index]
                is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

                n_users_evaluated += 1

                for cutoff in self.cutoff_list:

                    results_current_cutoff = results_dict[cutoff]

                    is_relevant_current_cutoff = is_relevant[0:cutoff]
                    recommended_items_current_cutoff = recommended_items[0:cutoff]

                    results_current_cutoff[EvaluatorMetrics.ROC_AUC.value] += roc_auc(is_relevant_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.PRECISION.value] += precision(is_relevant_current_cutoff)
                    results_current_cutoff[
                        EvaluatorMetrics.PRECISION_RECALL_MIN_DEN.value] += precision_recall_min_denominator(
                        is_relevant_current_cutoff, len(relevant_items))
                    results_current_cutoff[EvaluatorMetrics.RECALL.value] += recall(is_relevant_current_cutoff,
                                                                                    relevant_items)
                    results_current_cutoff[EvaluatorMetrics.NDCG.value] += ndcg(recommended_items_current_cutoff,
                                                                                relevant_items,
                                                                                relevance=self.get_user_test_ratings(
                                                                                    test_user), at=cutoff)
                    results_current_cutoff[EvaluatorMetrics.HIT_RATE.value] += is_relevant_current_cutoff.sum()
                    results_current_cutoff[EvaluatorMetrics.ARHR.value] += arhr(is_relevant_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.RMSE.value] += user_rmse

                    results_current_cutoff[EvaluatorMetrics.MRR.value].add_recommendations(is_relevant_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.MAP.value].add_recommendations(is_relevant_current_cutoff,
                                                                                           relevant_items)
                    results_current_cutoff[EvaluatorMetrics.NOVELTY.value].add_recommendations(
                        recommended_items_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.AVERAGE_POPULARITY.value].add_recommendations(
                        recommended_items_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.DIVERSITY_GINI.value].add_recommendations(
                        recommended_items_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.SHANNON_ENTROPY.value].add_recommendations(
                        recommended_items_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.COVERAGE_ITEM.value].add_recommendations(
                        recommended_items_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.COVERAGE_USER.value].add_recommendations(
                        recommended_items_current_cutoff, test_user)
                    results_current_cutoff[EvaluatorMetrics.DIVERSITY_MEAN_INTER_LIST.value].add_recommendations(
                        recommended_items_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.DIVERSITY_HERFINDAHL.value].add_recommendations(
                        recommended_items_current_cutoff)

                    if EvaluatorMetrics.DIVERSITY_SIMILARITY.value in results_current_cutoff:
                        results_current_cutoff[EvaluatorMetrics.DIVERSITY_SIMILARITY.value].add_recommendations(
                            recommended_items_current_cutoff)

                if time.time() - start_time_print > 30 or n_users_evaluated == len(self.usersToEvaluate):
                    elapsed_time = time.time() - start_time
                    new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)

                    self._print("Processed {} ( {:.2f}% ) in {:.2f} {}. Users per second: {:.0f}".format(
                        n_users_evaluated,
                        100.0 * float(n_users_evaluated) / len(self.usersToEvaluate),
                        new_time_value, new_time_unit,
                        float(n_users_evaluated) / elapsed_time))

                    sys.stdout.flush()
                    sys.stderr.flush()

                    start_time_print = time.time()

        return results_dict, n_users_evaluated

    def evaluateRecommender(self, recommender_object):
        """
        :param recommender_object: the trained recommender object, a BaseRecommender subclass
        :param URM_test_list: list of URMs to test the recommender against, or a single URM object
        :param cutoff_list: list of cutoffs to be use to report the scores, or a single cutoff
        """

        if self.ignore_items_flag:
            recommender_object.set_items_to_ignore(self.ignore_items_ID)

        results_dict, n_users_evaluated = self._run_evaluation_on_selected_users(recommender_object,
                                                                                 self.usersToEvaluate)

        if (n_users_evaluated > 0):

            for cutoff in self.cutoff_list:

                results_current_cutoff = results_dict[cutoff]

                for key in results_current_cutoff.keys():

                    value = results_current_cutoff[key]

                    if isinstance(value, Metrics_Object):
                        results_current_cutoff[key] = value.get_metric_value()
                    else:
                        results_current_cutoff[key] = value / n_users_evaluated

                precision_ = results_current_cutoff[EvaluatorMetrics.PRECISION.value]
                recall_ = results_current_cutoff[EvaluatorMetrics.RECALL.value]

                if precision_ + recall_ != 0:
                    # F1 micro averaged: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.104.8244&rep=rep1&type=pdf
                    results_current_cutoff[EvaluatorMetrics.F1.value] = 2 * (precision_ * recall_) / (
                            precision_ + recall_)
        else:
            self._print("WARNING: No users had a sufficient number of relevant items")

        results_run_string = get_result_string(results_dict)

        if self.ignore_items_flag:
            recommender_object.reset_items_to_ignore()

        return (results_dict, results_run_string)
    
    
    
# metrics.py 
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: Maurizio Ferrari Dacrema, Massimo Quadrana
"""

import numpy as np
import unittest

class Metrics_Object(object):
    """
    Abstract class that should be used as superclass of all metrics requiring an object, therefore a state, to be computed
    """
    def __init__(self):
        pass

    def add_recommendations(self, recommended_items_ids):
        raise NotImplementedError()

    def get_metric_value(self):
        raise NotImplementedError()

    def merge_with_other(self, other_metric_object):
        raise NotImplementedError()


class Coverage_Item(Metrics_Object):
    """
    Item coverage represents the percentage of the overall items which were recommended
    https://gab41.lab41.org/recommender-systems-its-not-all-about-the-accuracy-562c7dceeaff
    """

    def __init__(self, n_items, ignore_items):
        super(Coverage_Item, self).__init__()
        self.recommended_mask = np.zeros(n_items, dtype=np.bool)
        self.n_ignore_items = len(ignore_items)

    def add_recommendations(self, recommended_items_ids):
        if len(recommended_items_ids) > 0:
            self.recommended_mask[recommended_items_ids] = True

    def get_metric_value(self):
        return self.recommended_mask.sum()/(len(self.recommended_mask)-self.n_ignore_items)


    def merge_with_other(self, other_metric_object):
        assert other_metric_object is Coverage_Item, "Coverage_Item: attempting to merge with a metric object of different type"

        self.recommended_mask = np.logical_or(self.recommended_mask, other_metric_object.recommended_mask)

class Coverage_User(Metrics_Object):
    """
    User coverage represents the percentage of the overall users for which we can make recommendations.
    If there is at least one recommendation the user is considered as covered
    https://gab41.lab41.org/recommender-systems-its-not-all-about-the-accuracy-562c7dceeaff
    """

    def __init__(self, n_users, ignore_users):
        super(Coverage_User, self).__init__()
        self.users_mask = np.zeros(n_users, dtype=np.bool)
        self.n_ignore_users = len(ignore_users)

    def add_recommendations(self, recommended_items_ids, user_id):
        self.users_mask[user_id] = len(recommended_items_ids)>0

    def get_metric_value(self):
        return self.users_mask.sum()/(len(self.users_mask)-self.n_ignore_users)

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is Coverage_User, "Coverage_User: attempting to merge with a metric object of different type"

        self.users_mask = np.logical_or(self.users_mask, other_metric_object.users_mask)

class MAP(Metrics_Object):
    """
    Mean Average Precision, defined as the mean of the AveragePrecision over all users
    """

    def __init__(self):
        super(MAP, self).__init__()
        self.cumulative_AP = 0.0
        self.n_users = 0

    def add_recommendations(self, is_relevant, pos_items):
        self.cumulative_AP += average_precision(is_relevant, pos_items)
        self.n_users += 1

    def get_metric_value(self):
        return self.cumulative_AP/self.n_users

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is MAP, "MAP: attempting to merge with a metric object of different type"

        self.cumulative_AP += other_metric_object.cumulative_AP
        self.n_users += other_metric_object.n_users

class MRR(Metrics_Object):
    """
    Mean Reciprocal Rank, defined as the mean of the Reciprocal Rank over all users
    """

    def __init__(self):
        super(MRR, self).__init__()
        self.cumulative_RR = 0.0
        self.n_users = 0

    def add_recommendations(self, is_relevant):
        self.cumulative_RR += rr(is_relevant)
        self.n_users += 1

    def get_metric_value(self):
        return self.cumulative_RR/self.n_users

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is MAP, "MRR: attempting to merge with a metric object of different type"

        self.cumulative_RR += other_metric_object.cumulative_RR
        self.n_users += other_metric_object.n_users

class Gini_Diversity(Metrics_Object):
    """
    Gini diversity index, computed from the Gini Index but with inverted range, such that high values mean higher diversity
    This implementation ignores zero-occurrence items
    # From https://github.com/oliviaguest/gini
    # based on bottom eq: http://www.statsdirect.com/help/content/image/stat0206_wmf.gif
    # from: http://www.statsdirect.com/help/default.htm#nonparametric_methods/gini.htm
    #
    # http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.459.8174&rep=rep1&type=pdf
    """

    def __init__(self, n_items, ignore_items):
        super(Gini_Diversity, self).__init__()
        self.recommended_counter = np.zeros(n_items, dtype=np.float)
        self.ignore_items = ignore_items.astype(np.int).copy()

    def add_recommendations(self, recommended_items_ids):
        if len(recommended_items_ids) > 0:
            self.recommended_counter[recommended_items_ids] += 1

    def get_metric_value(self):

        recommended_counter = self.recommended_counter.copy()

        recommended_counter_mask = np.ones_like(recommended_counter, dtype = np.bool)
        recommended_counter_mask[self.ignore_items] = False
        recommended_counter_mask[recommended_counter == 0] = False

        recommended_counter = recommended_counter[recommended_counter_mask]

        n_items = len(recommended_counter)

        recommended_counter_sorted = np.sort(recommended_counter)       # values must be sorted
        index = np.arange(1, n_items+1)                                 # index per array element

        #gini_index = (np.sum((2 * index - n_items  - 1) * recommended_counter_sorted)) / (n_items * np.sum(recommended_counter_sorted))
        gini_diversity = 2*np.sum((n_items + 1 - index)/(n_items+1) * recommended_counter_sorted/np.sum(recommended_counter_sorted))

        return gini_diversity

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is Gini_Diversity, "Gini_Diversity: attempting to merge with a metric object of different type"

        self.recommended_counter += other_metric_object.recommended_counter

class Diversity_Herfindahl(Metrics_Object):
    """
    The Herfindahl index is also known as Concentration index, it is used in economy to determine whether the market quotas
    are such that an excessive concentration exists. It is here used as a diversity index, if high means high diversity.
    It is known to have a small value range in recommender systems, between 0.9 and 1.0
    The Herfindahl index is a function of the square of the probability an item has been recommended to any user, hence
    The Herfindahl index is equivalent to MeanInterList diversity as they measure the same quantity.
    # http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.459.8174&rep=rep1&type=pdf
    """

    def __init__(self, n_items, ignore_items):
        super(Diversity_Herfindahl, self).__init__()
        self.recommended_counter = np.zeros(n_items, dtype=np.float)
        self.ignore_items = ignore_items.astype(np.int).copy()

    def add_recommendations(self, recommended_items_ids):
        if len(recommended_items_ids) > 0:
            self.recommended_counter[recommended_items_ids] += 1

    def get_metric_value(self):

        recommended_counter = self.recommended_counter.copy()

        recommended_counter_mask = np.ones_like(recommended_counter, dtype = np.bool)
        recommended_counter_mask[self.ignore_items] = False

        recommended_counter = recommended_counter[recommended_counter_mask]

        if recommended_counter.sum() != 0:
            herfindahl_index = 1 - np.sum((recommended_counter / recommended_counter.sum()) ** 2)
        else:
            herfindahl_index = np.nan

        return herfindahl_index

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is Diversity_Herfindahl, "Diversity_Herfindahl: attempting to merge with a metric object of different type"

        self.recommended_counter += other_metric_object.recommended_counter

class Shannon_Entropy(Metrics_Object):
    """
    Shannon Entropy is a well known metric to measure the amount of information of a certain string of data.
    Here is applied to the global number of times an item has been recommended.
    It has a lower bound and can reach values over 12.0 for random recommenders.
    A high entropy means that the distribution is random uniform across all users.
    Note that while a random uniform distribution
    (hence all items with SIMILAR number of occurrences)
    will be highly diverse and have high entropy, a perfectly uniform distribution
    (hence all items with EXACTLY IDENTICAL number of occurrences)
    will have 0.0 entropy while being the most diverse possible.
    """

    def __init__(self, n_items, ignore_items):
        super(Shannon_Entropy, self).__init__()
        self.recommended_counter = np.zeros(n_items, dtype=np.float)
        self.ignore_items = ignore_items.astype(np.int).copy()

    def add_recommendations(self, recommended_items_ids):
        if len(recommended_items_ids) > 0:
            self.recommended_counter[recommended_items_ids] += 1

    def get_metric_value(self):

        assert np.all(self.recommended_counter >= 0.0), "Shannon_Entropy: self.recommended_counter contains negative counts"

        recommended_counter = self.recommended_counter.copy()

        # Ignore from the computation both ignored items and items with zero occurrence.
        # Zero occurrence items will have zero probability and will not change the result, butt will generate nans if used in the log
        recommended_counter_mask = np.ones_like(recommended_counter, dtype = np.bool)
        recommended_counter_mask[self.ignore_items] = False
        recommended_counter_mask[recommended_counter == 0] = False

        recommended_counter = recommended_counter[recommended_counter_mask]

        n_recommendations = recommended_counter.sum()

        recommended_probability = recommended_counter/n_recommendations

        shannon_entropy = -np.sum(recommended_probability * np.log2(recommended_probability))

        return shannon_entropy

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is Gini_Diversity, "Shannon_Entropy: attempting to merge with a metric object of different type"

        assert np.all(self.recommended_counter >= 0.0), "Shannon_Entropy: self.recommended_counter contains negative counts"
        assert np.all(other_metric_object.recommended_counter >= 0.0), "Shannon_Entropy: other.recommended_counter contains negative counts"

        self.recommended_counter += other_metric_object.recommended_counter


import scipy.sparse as sps

class Novelty(Metrics_Object):
    """
    Novelty measures how "novel" a recommendation is in terms of how popular the item was in the train set.
    Due to this definition, the novelty of a cold item (i.e. with no interactions in the train set) is not defined,
    in this implementation cold items are ignored and their contribution to the novelty is 0.
    A recommender with high novelty will be able to recommend also long queue (i.e. unpopular) items.
    Mean self-information  (Zhou 2010)
    """

    def __init__(self, URM_train):
        super(Novelty, self).__init__()

        URM_train = sps.csc_matrix(URM_train)
        URM_train.eliminate_zeros()
        self.item_popularity = np.ediff1d(URM_train.indptr)

        self.novelty = 0.0
        self.n_evaluated_users = 0
        self.n_items = len(self.item_popularity)
        self.n_interactions = self.item_popularity.sum()


    def add_recommendations(self, recommended_items_ids):

        self.n_evaluated_users += 1

        if len(recommended_items_ids)>0:
            recommended_items_popularity = self.item_popularity[recommended_items_ids]

            probability = recommended_items_popularity/self.n_interactions
            probability = probability[probability!=0]

            self.novelty += np.sum(-np.log2(probability)/self.n_items)

    def get_metric_value(self):

        if self.n_evaluated_users == 0:
            return 0.0

        return self.novelty/self.n_evaluated_users

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is Novelty, "Novelty: attempting to merge with a metric object of different type"

        self.novelty = self.novelty + other_metric_object.novelty
        self.n_evaluated_users = self.n_evaluated_users + other_metric_object.n_evaluated_users

class AveragePopularity(Metrics_Object):
    """
    Average popularity the recommended items have in the train data.
    The popularity is normalized by setting as 1 the item with the highest popularity in the train data
    """

    def __init__(self, URM_train):
        super(AveragePopularity, self).__init__()

        URM_train = sps.csc_matrix(URM_train)
        URM_train.eliminate_zeros()
        item_popularity = np.ediff1d(URM_train.indptr)


        self.cumulative_popularity = 0.0
        self.n_evaluated_users = 0
        self.n_items = URM_train.shape[0]
        self.n_interactions = item_popularity.sum()

        self.item_popularity_normalized = item_popularity/item_popularity.max()

    def add_recommendations(self, recommended_items_ids):

        self.n_evaluated_users += 1

        if len(recommended_items_ids)>0:
            recommended_items_popularity = self.item_popularity_normalized[recommended_items_ids]

            self.cumulative_popularity += np.sum(recommended_items_popularity)/len(recommended_items_ids)

    def get_metric_value(self):

        if self.n_evaluated_users == 0:
            return 0.0

        return self.cumulative_popularity/self.n_evaluated_users

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is Novelty, "AveragePopularity: attempting to merge with a metric object of different type"

        self.cumulative_popularity = self.cumulative_popularity + other_metric_object.cumulative_popularity
        self.n_evaluated_users = self.n_evaluated_users + other_metric_object.n_evaluated_users


class Diversity_similarity(Metrics_Object):
    """
    Intra list diversity computes the diversity of items appearing in the recommendations received by each single user, by using an item_diversity_matrix.
    It can be used, for example, to compute the diversity in terms of features for a collaborative recommender.
    A content-based recommender will have low IntraList diversity if that is computed on the same features the recommender uses.
    A TopPopular recommender may exhibit high IntraList diversity.
    """

    def __init__(self, item_diversity_matrix):
        super(Diversity_similarity, self).__init__()

        assert np.all(item_diversity_matrix >= 0.0) and np.all(item_diversity_matrix <= 1.0), \
            "item_diversity_matrix contains value greated than 1.0 or lower than 0.0"

        self.item_diversity_matrix = item_diversity_matrix

        self.n_evaluated_users = 0
        self.diversity = 0.0


    def add_recommendations(self, recommended_items_ids):

        current_recommended_items_diversity = 0.0

        for item_index in range(len(recommended_items_ids)-1):

            item_id = recommended_items_ids[item_index]

            item_other_diversity = self.item_diversity_matrix[item_id, recommended_items_ids]
            item_other_diversity[item_index] = 0.0

            current_recommended_items_diversity += np.sum(item_other_diversity)


        self.diversity += current_recommended_items_diversity/(len(recommended_items_ids)*(len(recommended_items_ids)-1))

        self.n_evaluated_users += 1


    def get_metric_value(self):

        if self.n_evaluated_users == 0:
            return 0.0

        return self.diversity/self.n_evaluated_users

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is Diversity_similarity, "Diversity: attempting to merge with a metric object of different type"

        self.diversity = self.diversity + other_metric_object.diversity
        self.n_evaluated_users = self.n_evaluated_users + other_metric_object.n_evaluated_users


class Diversity_MeanInterList(Metrics_Object):
    """
    MeanInterList diversity measures the uniqueness of different users' recommendation lists.
    It can be used to measure how "diversified" are the recommendations different users receive.
    While the original proposal called this metric "Personalization", we do not use this name since the highest MeanInterList diversity
    is exhibited by a non personalized Random recommender.
    It can be demonstrated that this metric does not require to compute the common items all possible couples of users have in common
    but rather it is only sensitive to the total amount of time each item has been recommended.
    MeanInterList diversity is a function of the square of the probability an item has been recommended to any user, hence
    MeanInterList diversity is equivalent to the Herfindahl index as they measure the same quantity.
    A TopPopular recommender that does not remove seen items will have 0.0 MeanInterList diversity.
    pag. 3, http://www.pnas.org/content/pnas/107/10/4511.full.pdf
    @article{zhou2010solving,
      title={Solving the apparent diversity-accuracy dilemma of recommender systems},
      author={Zhou, Tao and Kuscsik, Zolt{\'a}n and Liu, Jian-Guo and Medo, Mat{\'u}{\v{s}} and Wakeling, Joseph Rushton and Zhang, Yi-Cheng},
      journal={Proceedings of the National Academy of Sciences},
      volume={107},
      number={10},
      pages={4511--4515},
      year={2010},
      publisher={National Acad Sciences}
    }
    # The formula is diversity_cumulative += 1 - common_recommendations(user1, user2)/cutoff
    # for each couple of users, except the diagonal. It is VERY computationally expensive
    # We can move the 1 and cutoff outside of the summation. Remember to exclude the diagonal
    # co_counts = URM_predicted.dot(URM_predicted.T)
    # co_counts[np.arange(0, n_user, dtype=np.int):np.arange(0, n_user, dtype=np.int)] = 0
    # diversity = (n_user**2 - n_user) - co_counts.sum()/self.cutoff
    # If we represent the summation of co_counts separating it for each item, we will have:
    # co_counts.sum() = co_counts_item1.sum()  + co_counts_item2.sum() ...
    # If we know how many times an item has been recommended, co_counts_item1.sum() can be computed as how many couples of
    # users have item1 in common. If item1 has been recommended n times, the number of couples is n*(n-1)
    # Therefore we can compute co_counts.sum() value as:
    # np.sum(np.multiply(item-occurrence, item-occurrence-1))
    # The naive implementation URM_predicted.dot(URM_predicted.T) might require an hour of computation
    # The last implementation has a negligible computational time even for very big datasets
    """

    def __init__(self, n_items, cutoff):
        super(Diversity_MeanInterList, self).__init__()

        self.recommended_counter = np.zeros(n_items, dtype=np.float)

        self.n_evaluated_users = 0
        self.n_items = n_items
        self.diversity = 0.0
        self.cutoff = cutoff


    def add_recommendations(self, recommended_items_ids):

        assert len(recommended_items_ids) <= self.cutoff, "Diversity_MeanInterList: recommended list is contains more elements than cutoff"

        self.n_evaluated_users += 1

        if len(recommended_items_ids) > 0:
            self.recommended_counter[recommended_items_ids] += 1

    def get_metric_value(self):

        # Requires to compute the number of common elements for all couples of users
        if self.n_evaluated_users == 0:
            return 1.0

        cooccurrences_cumulative = np.sum(self.recommended_counter**2) - self.n_evaluated_users*self.cutoff

        # All user combinations except diagonal
        all_user_couples_count = self.n_evaluated_users**2 - self.n_evaluated_users

        diversity_cumulative = all_user_couples_count - cooccurrences_cumulative/self.cutoff

        self.diversity = diversity_cumulative/all_user_couples_count

        return self.diversity

    def get_theoretical_max(self):

        global_co_occurrence_count = (self.n_evaluated_users*self.cutoff)**2/self.n_items - self.n_evaluated_users*self.cutoff

        mild = 1 - 1/(self.n_evaluated_users**2 - self.n_evaluated_users)*(global_co_occurrence_count/self.cutoff)

        return mild

    def merge_with_other(self, other_metric_object):

        assert other_metric_object is Diversity_MeanInterList, "Diversity_MeanInterList: attempting to merge with a metric object of different type"

        assert np.all(self.recommended_counter >= 0.0), "Diversity_MeanInterList: self.recommended_counter contains negative counts"
        assert np.all(other_metric_object.recommended_counter >= 0.0), "Diversity_MeanInterList: other.recommended_counter contains negative counts"

        self.recommended_counter += other_metric_object.recommended_counter
        self.n_evaluated_users += other_metric_object.n_evaluated_users

def roc_auc(is_relevant):

    ranks = np.arange(len(is_relevant))
    pos_ranks = ranks[is_relevant]
    neg_ranks = ranks[~is_relevant]
    auc_score = 0.0

    if len(neg_ranks) == 0:
        return 1.0

    if len(pos_ranks) > 0:
        for pos_pred in pos_ranks:
            auc_score += np.sum(pos_pred < neg_ranks, dtype=np.float32)
        auc_score /= (pos_ranks.shape[0] * neg_ranks.shape[0])

    assert 0 <= auc_score <= 1, auc_score
    return auc_score

def arhr(is_relevant):
    # average reciprocal hit-rank (ARHR) of all relevant items
    # As opposed to MRR, ARHR takes into account all relevant items and not just the first
    # pag 17
    # http://glaros.dtc.umn.edu/gkhome/fetch/papers/itemrsTOIS04.pdf
    # https://emunix.emich.edu/~sverdlik/COSC562/ItemBasedTopTen.pdf

    p_reciprocal = 1/np.arange(1,len(is_relevant)+1, 1.0, dtype=np.float64)
    arhr_score = is_relevant.dot(p_reciprocal)

    #assert 0 <= arhr_score <= p_reciprocal.sum(), "arhr_score {} should be between 0 and {}".format(arhr_score, p_reciprocal.sum())
    assert not np.isnan(arhr_score), "ARHR is NaN"
    return arhr_score

def precision(is_relevant):

    if len(is_relevant) == 0:
        precision_score = 0.0
    else:
        precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    assert 0 <= precision_score <= 1, precision_score
    return precision_score


def precision_recall_min_denominator(is_relevant, n_test_items):

    if len(is_relevant) == 0:
        precision_score = 0.0
    else:
        precision_score = np.sum(is_relevant, dtype=np.float32) / min(n_test_items, len(is_relevant))

    assert 0 <= precision_score <= 1, precision_score
    return precision_score

def rmse(all_items_predicted_ratings, relevant_items, relevant_items_rating):

    # Important, some items will have -np.inf score and are treated as if they did not exist

    # RMSE with test items
    relevant_items_error = (all_items_predicted_ratings[relevant_items]-relevant_items_rating)**2

    finite_prediction_mask = np.isfinite(relevant_items_error)

    if finite_prediction_mask.sum() == 0:
        rmse = np.nan

    else:
        relevant_items_error = relevant_items_error[finite_prediction_mask]

        squared_error = np.sum(relevant_items_error)

        # # Second the RMSE against all non-test items assumed having true rating 0
        # # In order to avoid the need of explicitly indexing all non-relevant items, use a difference
        # squared_error += np.sum(all_items_predicted_ratings[np.isfinite(all_items_predicted_ratings)]**2) - \
        #                  np.sum(all_items_predicted_ratings[relevant_items][np.isfinite(all_items_predicted_ratings[relevant_items])]**2)

        mean_squared_error = squared_error/finite_prediction_mask.sum()
        rmse = np.sqrt(mean_squared_error)

    return rmse


def recall(is_relevant, pos_items):

    recall_score = np.sum(is_relevant, dtype=np.float32) / pos_items.shape[0]

    assert 0 <= recall_score <= 1, recall_score
    return recall_score


def rr(is_relevant):
    # reciprocal rank of the FIRST relevant item in the ranked list (0 if none)

    ranks = np.arange(1, len(is_relevant) + 1)[is_relevant]

    if len(ranks) > 0:
        return 1. / ranks[0]
    else:
        return 0.0


def average_precision(is_relevant, pos_items):

    if len(is_relevant) == 0:
        a_p = 0.0
    else:
        p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
        a_p = np.sum(p_at_k) / np.min([pos_items.shape[0], is_relevant.shape[0]])

    assert 0 <= a_p <= 1, a_p
    return a_p


def ndcg(ranked_list, pos_items, relevance=None, at=None):

    if relevance is None:
        relevance = np.ones_like(pos_items)
    assert len(relevance) == pos_items.shape[0]

    # Create a dictionary associating item_id to its relevance
    # it2rel[item] -> relevance[item]
    it2rel = {it: r for it, r in zip(pos_items, relevance)}

    # Creates array of length "at" with the relevance associated to the item in that position
    rank_scores = np.asarray([it2rel.get(it, 0.0) for it in ranked_list[:at]], dtype=np.float32)

    # IDCG has all relevances to 1, up to the number of items in the test set
    ideal_dcg = dcg(np.sort(relevance)[::-1])

    # DCG uses the relevance of the recommended items
    rank_dcg = dcg(rank_scores)

    if rank_dcg == 0.0:
        return 0.0

    ndcg_ = rank_dcg / ideal_dcg
    # assert 0 <= ndcg_ <= 1, (rank_dcg, ideal_dcg, ndcg_)
    return ndcg_


def dcg(scores):
    return np.sum(np.divide(np.power(2, scores) - 1, np.log(np.arange(scores.shape[0], dtype=np.float32) + 2)),
                  dtype=np.float32)


metrics = ['AUC', 'Precision' 'Recall', 'MAP', 'NDCG']


def pp_metrics(metric_names, metric_values, metric_at):
    """
    Pretty-prints metric values
    :param metrics_arr:
    :return:
    """
    assert len(metric_names) == len(metric_values)
    if isinstance(metric_at, int):
        metric_at = [metric_at] * len(metric_values)
    return ' '.join(['{}: {:.4f}'.format(mname, mvalue) if mcutoff is None or mcutoff == 0 else
                     '{}@{}: {:.4f}'.format(mname, mcutoff, mvalue)
                     for mname, mcutoff, mvalue in zip(metric_names, metric_at, metric_values)])


class TestAUC(unittest.TestCase):
    def runTest(self):
        pos_items = np.asarray([2, 4])
        ranked_list = np.asarray([1, 2, 3, 4, 5])
        self.assertTrue(np.allclose(roc_auc(ranked_list, pos_items),
                                    (2. / 3 + 1. / 3) / 2))


class TestRecall(unittest.TestCase):
    def runTest(self):
        pos_items = np.asarray([2, 4, 5, 10])
        ranked_list_1 = np.asarray([1, 2, 3, 4, 5])
        ranked_list_2 = np.asarray([10, 5, 2, 4, 3])
        ranked_list_3 = np.asarray([1, 3, 6, 7, 8])
        self.assertTrue(np.allclose(recall(ranked_list_1, pos_items), 3. / 4))
        self.assertTrue(np.allclose(recall(ranked_list_2, pos_items), 1.0))
        self.assertTrue(np.allclose(recall(ranked_list_3, pos_items), 0.0))

        thresholds = [1, 2, 3, 4, 5]
        values = [0.0, 1. / 4, 1. / 4, 2. / 4, 3. / 4]
        for at, val in zip(thresholds, values):
            self.assertTrue(np.allclose(np.asarray(recall(ranked_list_1, pos_items, at=at)), val))


class TestPrecision(unittest.TestCase):
    def runTest(self):
        pos_items = np.asarray([2, 4, 5, 10])
        ranked_list_1 = np.asarray([1, 2, 3, 4, 5])
        ranked_list_2 = np.asarray([10, 5, 2, 4, 3])
        ranked_list_3 = np.asarray([1, 3, 6, 7, 8])
        self.assertTrue(np.allclose(precision(ranked_list_1, pos_items), 3. / 5))
        self.assertTrue(np.allclose(precision(ranked_list_2, pos_items), 4. / 5))
        self.assertTrue(np.allclose(precision(ranked_list_3, pos_items), 0.0))

        thresholds = [1, 2, 3, 4, 5]
        values = [0.0, 1. / 2, 1. / 3, 2. / 4, 3. / 5]
        for at, val in zip(thresholds, values):
            self.assertTrue(np.allclose(np.asarray(precision(ranked_list_1, pos_items, at=at)), val))


class TestRR(unittest.TestCase):
    def runTest(self):
        pos_items = np.asarray([2, 4, 5, 10])
        ranked_list_1 = np.asarray([1, 2, 3, 4, 5])
        ranked_list_2 = np.asarray([10, 5, 2, 4, 3])
        ranked_list_3 = np.asarray([1, 3, 6, 7, 8])
        self.assertTrue(np.allclose(rr(ranked_list_1, pos_items), 1. / 2))
        self.assertTrue(np.allclose(rr(ranked_list_2, pos_items), 1.))
        self.assertTrue(np.allclose(rr(ranked_list_3, pos_items), 0.0))

        thresholds = [1, 2, 3, 4, 5]
        values = [0.0, 1. / 2, 1. / 2, 1. / 2, 1. / 2]
        for at, val in zip(thresholds, values):
            self.assertTrue(np.allclose(np.asarray(rr(ranked_list_1, pos_items, at=at)), val))


class TestMAP(unittest.TestCase):
    def runTest(self):
        pos_items = np.asarray([2, 4, 5, 10])
        ranked_list_1 = np.asarray([1, 2, 3, 4, 5])
        ranked_list_2 = np.asarray([10, 5, 2, 4, 3])
        ranked_list_3 = np.asarray([1, 3, 6, 7, 8])
        ranked_list_4 = np.asarray([11, 12, 13, 14, 15, 16, 2, 4, 5, 10])
        ranked_list_5 = np.asarray([2, 11, 12, 13, 14, 15, 4, 5, 10, 16])
        self.assertTrue(np.allclose(map(ranked_list_1, pos_items), (1. / 2 + 2. / 4 + 3. / 5) / 4))
        self.assertTrue(np.allclose(map(ranked_list_2, pos_items), 1.0))
        self.assertTrue(np.allclose(map(ranked_list_3, pos_items), 0.0))
        self.assertTrue(np.allclose(map(ranked_list_4, pos_items), (1. / 7 + 2. / 8 + 3. / 9 + 4. / 10) / 4))
        self.assertTrue(np.allclose(map(ranked_list_5, pos_items), (1. + 2. / 7 + 3. / 8 + 4. / 9) / 4))

        thresholds = [1, 2, 3, 4, 5]
        values = [
            0.0,
            1. / 2 / 2,
            1. / 2 / 3,
            (1. / 2 + 2. / 4) / 4,
            (1. / 2 + 2. / 4 + 3. / 5) / 4
        ]
        for at, val in zip(thresholds, values):
            self.assertTrue(np.allclose(np.asarray(map(ranked_list_1, pos_items, at)), val))


class TestNDCG(unittest.TestCase):
    def runTest(self):
        pos_items = np.asarray([2, 4, 5, 10])
        pos_relevances = np.asarray([5, 4, 3, 2])
        ranked_list_1 = np.asarray([1, 2, 3, 4, 5])  # rel = 0, 5, 0, 4, 3
        ranked_list_2 = np.asarray([10, 5, 2, 4, 3])  # rel = 2, 3, 5, 4, 0
        ranked_list_3 = np.asarray([1, 3, 6, 7, 8])  # rel = 0, 0, 0, 0, 0
        idcg = ((2 ** 5 - 1) / np.log(2) +
                (2 ** 4 - 1) / np.log(3) +
                (2 ** 3 - 1) / np.log(4) +
                (2 ** 2 - 1) / np.log(5))
        self.assertTrue(np.allclose(dcg(np.sort(pos_relevances)[::-1]), idcg))
        self.assertTrue(np.allclose(ndcg(ranked_list_1, pos_items, pos_relevances),
                                    ((2 ** 5 - 1) / np.log(3) +
                                     (2 ** 4 - 1) / np.log(5) +
                                     (2 ** 3 - 1) / np.log(6)) / idcg))
        self.assertTrue(np.allclose(ndcg(ranked_list_2, pos_items, pos_relevances),
                                    ((2 ** 2 - 1) / np.log(2) +
                                     (2 ** 3 - 1) / np.log(3) +
                                     (2 ** 5 - 1) / np.log(4) +
                                     (2 ** 4 - 1) / np.log(5)) / idcg))
        self.assertTrue(np.allclose(ndcg(ranked_list_3, pos_items, pos_relevances), 0.0))


# if __name__ == '__main__':
#     unittest.main()


# seconds_to_biggest_unit.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 30/03/2019
@author: Maurizio Ferrari Dacrema
"""


def seconds_to_biggest_unit(time_in_seconds, data_array = None):

    conversion_factor = [
        ("sec", 60),
        ("min", 60),
        ("hour", 24),
        ("day", 365),
    ]

    terminate = False
    unit_index = 0

    new_time_value = time_in_seconds
    new_time_unit = "sec"

    while not terminate:

        next_time = new_time_value/conversion_factor[unit_index][1]

        if next_time >= 1.0:
            new_time_value = next_time

            if data_array is not None:
                data_array /= conversion_factor[unit_index][1]

            unit_index += 1
            new_time_unit = conversion_factor[unit_index][0]

        else:
            terminate = True


    if data_array is not None:
        return new_time_value, new_time_unit, data_array

    else:
        return new_time_value, new_time_unit


> **Hyperparams tuning**

In [6]:
# SearchAbstractClass.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 10/03/2018
@author: Maurizio Ferrari Dacrema
"""

import time, os, traceback
# from utils.Evaluation.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping
import numpy as np
# from utils.DataIO import DataIO

class SearchInputRecommenderArgs(object):


    def __init__(self,
                   # Dictionary of parameters needed by the constructor
                   CONSTRUCTOR_POSITIONAL_ARGS = None,
                   CONSTRUCTOR_KEYWORD_ARGS = None,

                   # List containing all positional arguments needed by the fit function
                   FIT_POSITIONAL_ARGS = None,
                   FIT_KEYWORD_ARGS = None
                   ):


          super(SearchInputRecommenderArgs, self).__init__()

          if CONSTRUCTOR_POSITIONAL_ARGS is None:
              CONSTRUCTOR_POSITIONAL_ARGS = []

          if CONSTRUCTOR_KEYWORD_ARGS is None:
              CONSTRUCTOR_KEYWORD_ARGS = {}

          if FIT_POSITIONAL_ARGS is None:
              FIT_POSITIONAL_ARGS = []

          if FIT_KEYWORD_ARGS is None:
              FIT_KEYWORD_ARGS = {}


          assert isinstance(CONSTRUCTOR_POSITIONAL_ARGS, list), "CONSTRUCTOR_POSITIONAL_ARGS must be a list"
          assert isinstance(CONSTRUCTOR_KEYWORD_ARGS, dict), "CONSTRUCTOR_KEYWORD_ARGS must be a dict"

          assert isinstance(FIT_POSITIONAL_ARGS, list), "FIT_POSITIONAL_ARGS must be a list"
          assert isinstance(FIT_KEYWORD_ARGS, dict), "FIT_KEYWORD_ARGS must be a dict"


          self.CONSTRUCTOR_POSITIONAL_ARGS = CONSTRUCTOR_POSITIONAL_ARGS
          self.CONSTRUCTOR_KEYWORD_ARGS = CONSTRUCTOR_KEYWORD_ARGS

          self.FIT_POSITIONAL_ARGS = FIT_POSITIONAL_ARGS
          self.FIT_KEYWORD_ARGS = FIT_KEYWORD_ARGS


    def copy(self):


        clone_object = SearchInputRecommenderArgs(
                            CONSTRUCTOR_POSITIONAL_ARGS = self.CONSTRUCTOR_POSITIONAL_ARGS.copy(),
                            CONSTRUCTOR_KEYWORD_ARGS = self.CONSTRUCTOR_KEYWORD_ARGS.copy(),
                            FIT_POSITIONAL_ARGS = self.FIT_POSITIONAL_ARGS.copy(),
                            FIT_KEYWORD_ARGS = self.FIT_KEYWORD_ARGS.copy()
                            )


        return clone_object



def _compute_avg_time_non_none_values(data_list):

    non_none_values = sum([value is not None for value in data_list])
    total_value = sum([value if value is not None else 0.0 for value in data_list])

    return total_value, \
           total_value/non_none_values if non_none_values != 0 else 0.0



def get_result_string_evaluate_on_validation(results_run_single_cutoff, n_decimals=7):

    output_str = ""

    for metric in results_run_single_cutoff.keys():
        output_str += "{}: {:.{n_decimals}f}, ".format(metric, results_run_single_cutoff[metric], n_decimals = n_decimals)

    return output_str



class SearchAbstractClass(object):

    ALGORITHM_NAME = "SearchAbstractClass"

    # Available values for the save_model attribute
    _SAVE_MODEL_VALUES = ["all", "best", "last", "no"]


    # Value to be assigned to invalid configuration or if an Exception is raised
    INVALID_CONFIG_VALUE = np.finfo(np.float16).max

    def __init__(self, recommender_class,
                 evaluator_validation = None,
                 evaluator_test = None,
                 verbose = True):

        super(SearchAbstractClass, self).__init__()

        self.recommender_class = recommender_class
        self.verbose = verbose
        self.log_file = None

        self.results_test_best = {}
        self.parameter_dictionary_best = {}

        self.evaluator_validation = evaluator_validation

        if evaluator_test is None:
            self.evaluator_test = None
        else:
            self.evaluator_test = evaluator_test


    def search(self, recommender_input_args,
               parameter_search_space,
               metric_to_optimize = "MAP",
               n_cases = None,
               output_folder_path = None,
               output_file_name_root = None,
               parallelize = False,
               save_model = "best",
               evaluate_on_test_each_best_solution = True,
               save_metadata = True,
               ):

        raise NotImplementedError("Function search not implemented for this class")


    def _set_search_attributes(self, recommender_input_args,
                               recommender_input_args_last_test,
                               metric_to_optimize,
                               output_folder_path,
                               output_file_name_root,
                               resume_from_saved,
                               save_metadata,
                               save_model,
                               evaluate_on_test_each_best_solution,
                               n_cases):


        if save_model not in self._SAVE_MODEL_VALUES:
           raise ValueError("{}: parameter save_model must be in '{}', provided was '{}'.".format(self.ALGORITHM_NAME, self._SAVE_MODEL_VALUES, save_model))

        self.output_folder_path = output_folder_path
        self.output_file_name_root = output_file_name_root

        # If directory does not exist, create
        if not os.path.exists(self.output_folder_path):
            os.makedirs(self.output_folder_path)

        self.log_file = open(self.output_folder_path + self.output_file_name_root + "_{}.txt".format(self.ALGORITHM_NAME), "a")

        if save_model == "last" and recommender_input_args_last_test is None:
            self._write_log("{}: parameter save_model is 'last' but no recommender_input_args_last_test provided, saving best model on train data alone.".format(self.ALGORITHM_NAME))
            save_model = "best"



        self.recommender_input_args = recommender_input_args
        self.recommender_input_args_last_test = recommender_input_args_last_test
        self.metric_to_optimize = metric_to_optimize
        self.save_model = save_model
        self.resume_from_saved = resume_from_saved
        self.save_metadata = save_metadata
        self.evaluate_on_test_each_best_solution = evaluate_on_test_each_best_solution

        self.model_counter = 0
        self._init_metadata_dict(n_cases = n_cases)

        if self.save_metadata:
            self.dataIO = DataIO(folder_path = self.output_folder_path)



    def _init_metadata_dict(self, n_cases):

        self.metadata_dict = {"algorithm_name_search": self.ALGORITHM_NAME,
                              "algorithm_name_recommender": self.recommender_class.RECOMMENDER_NAME,
                              "exception_list": [None]*n_cases,

                              "hyperparameters_list": [None]*n_cases,
                              "hyperparameters_best": None,
                              "hyperparameters_best_index": None,

                              "result_on_validation_list": [None]*n_cases,
                              "result_on_validation_best": None,
                              "result_on_test_list": [None]*n_cases,
                              "result_on_test_best": None,

                              "time_on_train_list": [None]*n_cases,
                              "time_on_train_total": 0.0,
                              "time_on_train_avg": 0.0,

                              "time_on_validation_list": [None]*n_cases,
                              "time_on_validation_total": 0.0,
                              "time_on_validation_avg": 0.0,

                              "time_on_test_list": [None]*n_cases,
                              "time_on_test_total": 0.0,
                              "time_on_test_avg": 0.0,

                              "result_on_last": None,
                              "time_on_last_train": None,
                              "time_on_last_test": None,
                              }


    def _print(self, string):

        if self.verbose:
            print(string)


    def _write_log(self, string):

        self._print(string)

        if self.log_file is not None:
            self.log_file.write(string)
            self.log_file.flush()


    def _fit_model(self, current_fit_parameters):

        start_time = time.time()

        # Construct a new recommender instance
        recommender_instance = self.recommender_class(*self.recommender_input_args.CONSTRUCTOR_POSITIONAL_ARGS,
                                                      **self.recommender_input_args.CONSTRUCTOR_KEYWORD_ARGS)


        self._print("{}: Testing config: {}".format(self.ALGORITHM_NAME, current_fit_parameters))


        recommender_instance.fit(*self.recommender_input_args.FIT_POSITIONAL_ARGS,
                                 **self.recommender_input_args.FIT_KEYWORD_ARGS,
                                 **current_fit_parameters)

        train_time = time.time() - start_time

        return recommender_instance, train_time



    def _evaluate_on_validation(self, current_fit_parameters):

        recommender_instance, train_time = self._fit_model(current_fit_parameters)

        start_time = time.time()

        # Evaluate recommender and get results for the first cutoff
        result_dict, _ = self.evaluator_validation.evaluateRecommender(recommender_instance)
        result_dict = result_dict[list(result_dict.keys())[0]]

        evaluation_time = time.time() - start_time

        result_string = get_result_string_evaluate_on_validation(result_dict, n_decimals=7)

        return result_dict, result_string, recommender_instance, train_time, evaluation_time



    def _evaluate_on_test(self, recommender_instance, current_fit_parameters_dict, print_log = True):

        start_time = time.time()

        # Evaluate recommender and get results for the first cutoff
        result_dict, result_string = self.evaluator_test.evaluateRecommender(recommender_instance)

        evaluation_test_time = time.time() - start_time

        if print_log:
            self._write_log("{}: Best config evaluated with evaluator_test. Config: {} - results:\n{}\n".format(self.ALGORITHM_NAME,
                                                                                                         current_fit_parameters_dict,
                                                                                                         result_string))

        return result_dict, result_string, evaluation_test_time



    def _evaluate_on_test_with_data_last(self):

        start_time = time.time()

        # Construct a new recommender instance
        recommender_instance = self.recommender_class(*self.recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS,
                                                      **self.recommender_input_args_last_test.CONSTRUCTOR_KEYWORD_ARGS)

        # Check if last was already evaluated
        if self.resume_from_saved:
            result_on_last_saved_flag = self.metadata_dict["result_on_last"] is not None and \
                                        self.metadata_dict["time_on_last_train"] is not None and \
                                        self.metadata_dict["time_on_last_test"] is not None

            if result_on_last_saved_flag:
                self._print("{}: Resuming '{}'... Result on last already available.".format(self.ALGORITHM_NAME, self.output_file_name_root))
                return



        self._print("{}: Evaluation with constructor data for final test. Using best config: {}".format(self.ALGORITHM_NAME, self.metadata_dict["hyperparameters_best"]))


        # Use the hyperparameters that have been saved
        assert self.metadata_dict["hyperparameters_best"] is not None, "{}: Best hyperparameters not available, the search might have failed.".format(self.ALGORITHM_NAME)
        fit_keyword_args = self.metadata_dict["hyperparameters_best"].copy()


        recommender_instance.fit(*self.recommender_input_args_last_test.FIT_POSITIONAL_ARGS,
                                 **fit_keyword_args)

        train_time = time.time() - start_time

        result_dict_test, result_string, evaluation_test_time = self._evaluate_on_test(recommender_instance, fit_keyword_args, print_log = False)

        self._write_log("{}: Best config evaluated with evaluator_test with constructor data for final test. Config: {} - results:\n{}\n".format(self.ALGORITHM_NAME,
                                                                                                                                          self.metadata_dict["hyperparameters_best"],
                                                                                                                                          result_string))

        self.metadata_dict["result_on_last"] = result_dict_test
        self.metadata_dict["time_on_last_train"] = train_time
        self.metadata_dict["time_on_last_test"] = evaluation_test_time

        if self.save_metadata:
            self.dataIO.save_data(data_dict_to_save = self.metadata_dict.copy(),
                                  file_name = self.output_file_name_root + "_metadata")

        if self.save_model in ["all", "best", "last"]:
            self._print("{}: Saving model in {}\n".format(self.ALGORITHM_NAME, self.output_folder_path + self.output_file_name_root))
            recommender_instance.save_model(self.output_folder_path, file_name =self.output_file_name_root + "_best_model_last")


    def _objective_function(self, current_fit_parameters_dict):

        try:

            self.metadata_dict["hyperparameters_list"][self.model_counter] = current_fit_parameters_dict.copy()

            result_dict, result_string, recommender_instance, train_time, evaluation_time = self._evaluate_on_validation(current_fit_parameters_dict)

            current_result = - result_dict[self.metric_to_optimize]

            # If the recommender uses Earlystopping, get the selected number of epochs
            if isinstance(recommender_instance, Incremental_Training_Early_Stopping):

                n_epochs_early_stopping_dict = recommender_instance.get_early_stopping_final_epochs_dict()
                current_fit_parameters_dict = current_fit_parameters_dict.copy()

                for epoch_label in n_epochs_early_stopping_dict.keys():

                    epoch_value = n_epochs_early_stopping_dict[epoch_label]
                    current_fit_parameters_dict[epoch_label] = epoch_value



            # Always save best model separately
            if self.save_model in ["all"]:
                self._print("{}: Saving model in {}\n".format(self.ALGORITHM_NAME, self.output_folder_path + self.output_file_name_root))
                recommender_instance.save_model(self.output_folder_path, file_name = self.output_file_name_root + "_model_{}".format(self.model_counter))


            if self.metadata_dict["result_on_validation_best"] is None:
                new_best_config_found = True
            else:
                best_solution_val = self.metadata_dict["result_on_validation_best"][self.metric_to_optimize]
                new_best_config_found = best_solution_val < result_dict[self.metric_to_optimize]


            if new_best_config_found:

                self._write_log("{}: New best config found. Config {}: {} - results: {}\n".format(self.ALGORITHM_NAME,
                                                                                           self.model_counter,
                                                                                           current_fit_parameters_dict,
                                                                                           result_string))

                if self.save_model in ["all", "best"]:
                    self._print("{}: Saving model in {}\n".format(self.ALGORITHM_NAME, self.output_folder_path + self.output_file_name_root))
                    recommender_instance.save_model(self.output_folder_path, file_name =self.output_file_name_root + "_best_model")


                if self.evaluator_test is not None and self.evaluate_on_test_each_best_solution:
                    result_dict_test, _, evaluation_test_time = self._evaluate_on_test(recommender_instance, current_fit_parameters_dict, print_log = True)


            else:
                self._write_log("{}: Config {} is suboptimal. Config: {} - results: {}\n".format(self.ALGORITHM_NAME,
                                                                                          self.model_counter,
                                                                                          current_fit_parameters_dict,
                                                                                          result_string))



            if current_result >= self.INVALID_CONFIG_VALUE:
                self._write_log("{}: WARNING! Config {} returned a value equal or worse than the default value to be assigned to invalid configurations."
                                " If no better valid configuration is found, this parameter search may produce an invalid result.\n")



            self.metadata_dict["result_on_validation_list"][self.model_counter] = result_dict.copy()

            self.metadata_dict["time_on_train_list"][self.model_counter] = train_time
            self.metadata_dict["time_on_validation_list"][self.model_counter] = evaluation_time

            self.metadata_dict["time_on_train_total"], self.metadata_dict["time_on_train_avg"] = \
                _compute_avg_time_non_none_values(self.metadata_dict["time_on_train_list"])
            self.metadata_dict["time_on_validation_total"], self.metadata_dict["time_on_validation_avg"] = \
                _compute_avg_time_non_none_values(self.metadata_dict["time_on_validation_list"])


            if new_best_config_found:
                self.metadata_dict["hyperparameters_best"] = current_fit_parameters_dict.copy()
                self.metadata_dict["hyperparameters_best_index"] = self.model_counter
                self.metadata_dict["result_on_validation_best"] = result_dict.copy()

                if self.evaluator_test is not None and self.evaluate_on_test_each_best_solution:
                    self.metadata_dict["result_on_test_best"] = result_dict_test.copy()
                    self.metadata_dict["result_on_test_list"][self.model_counter] = result_dict_test.copy()
                    self.metadata_dict["time_on_test_list"][self.model_counter] = evaluation_test_time

                    self.metadata_dict["time_on_test_total"], self.metadata_dict["time_on_test_avg"] = \
                        _compute_avg_time_non_none_values(self.metadata_dict["time_on_test_list"])


        except (KeyboardInterrupt, SystemExit) as e:
            # If getting a interrupt, terminate without saving the exception
            raise e

        except:
            # Catch any error: Exception, Tensorflow errors etc...

            traceback_string = traceback.format_exc()

            self._write_log("{}: Config {} Exception. Config: {} - Exception: {}\n".format(self.ALGORITHM_NAME,
                                                                                  self.model_counter,
                                                                                  current_fit_parameters_dict,
                                                                                  traceback_string))

            self.metadata_dict["exception_list"][self.model_counter] = traceback_string


            # Assign to this configuration the worst possible score
            # Being a minimization problem, set it to the max value of a float
            current_result = + self.INVALID_CONFIG_VALUE

            traceback.print_exc()



        if self.save_metadata:
            self.dataIO.save_data(data_dict_to_save = self.metadata_dict.copy(),
                                  file_name = self.output_file_name_root + "_metadata")

        self.model_counter += 1

        return current_result

In [7]:
# SearchBayesianSkopt.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 14/12/18
@author: Emanuele Chioso, Maurizio Ferrari Dacrema
"""

from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical

# from utils.ParameterTuning.SearchAbstractClass import SearchAbstractClass
import traceback


class SearchBayesianSkopt(SearchAbstractClass):

    ALGORITHM_NAME = "SearchBayesianSkopt"

    def __init__(self, recommender_class, evaluator_validation = None, evaluator_test = None, verbose = True):

        assert evaluator_validation is not None, "{}: evaluator_validation must be provided".format(self.ALGORITHM_NAME)

        super(SearchBayesianSkopt, self).__init__(recommender_class,
                                                  evaluator_validation = evaluator_validation,
                                                  evaluator_test = evaluator_test,
                                                  verbose = verbose)



    def _set_skopt_params(self, n_calls = 70,
                          n_random_starts = 20,
                          n_points = 10000,
                          n_jobs = 1,
                          # noise = 'gaussian',
                          noise = 1e-5,
                          acq_func = 'gp_hedge',
                          acq_optimizer = 'auto',
                          random_state = None,
                          verbose = True,
                          n_restarts_optimizer = 10,
                          xi = 0.01,
                          kappa = 1.96,
                          x0 = None,
                          y0 = None):
        """
        wrapper to change the params of the bayesian optimizator.
        for further details:
        https://scikit-optimize.github.io/#skopt.gp_minimize
        """
        self.n_point = n_points
        self.n_calls = n_calls
        self.n_random_starts = n_random_starts
        self.n_jobs = n_jobs
        self.acq_func = acq_func
        self.acq_optimizer = acq_optimizer
        self.random_state = random_state
        self.n_restarts_optimizer = n_restarts_optimizer
        self.verbose = verbose
        self.xi = xi
        self.kappa = kappa
        self.noise = noise
        self.x0 = x0
        self.y0 = y0



    def _resume_from_saved(self):

        try:
            self.metadata_dict = self.dataIO.load_data(file_name = self.output_file_name_root + "_metadata")

        except (KeyboardInterrupt, SystemExit) as e:
            # If getting a interrupt, terminate without saving the exception
            raise e

        except FileNotFoundError:
            self._write_log("{}: Resuming '{}' Failed, no such file exists.\n".format(self.ALGORITHM_NAME, self.output_file_name_root))
            self.resume_from_saved = False
            return None, None

        except Exception as e:
            self._write_log("{}: Resuming '{}' Failed, generic exception: {}.\n".format(self.ALGORITHM_NAME, self.output_file_name_root, str(e)))
            traceback.print_exc()
            self.resume_from_saved = False
            return None, None

        # Get hyperparameter list and corresponding result
        # Make sure that the hyperparameters only contain those given as input and not others like the number of epochs
        # selected by earlystopping
        hyperparameters_list_saved = self.metadata_dict['hyperparameters_list']
        result_on_validation_list_saved = self.metadata_dict['result_on_validation_list']

        hyperparameters_list_input = []
        result_on_validation_list_input = []

        # The hyperparameters are saved for all cases even if they throw an exception
        while self.model_counter<len(hyperparameters_list_saved) and hyperparameters_list_saved[self.model_counter] is not None:

            hyperparameters_config_saved = hyperparameters_list_saved[self.model_counter]

            hyperparameters_config_input = []

            # Add only those having a search space, in the correct ordering
            for index in range(len(self.hyperparams_names)):
                key = self.hyperparams_names[index]
                value_saved = hyperparameters_config_saved[key]

                # Check if single value categorical. It is aimed at intercepting
                # Hyperparameters that are chosen via early stopping and set them as the
                # maximum value as per hyperparameter search space. If not, the gp_minimize will return an error
                # as some values will be outside (lower) than the search space

                if isinstance(self.hyperparams_values[index], Categorical) and self.hyperparams_values[index].transformed_size == 1:
                    value_input = self.hyperparams_values[index].bounds[0]
                else:
                    value_input = value_saved

                hyperparameters_config_input.append(value_input)


            hyperparameters_list_input.append(hyperparameters_config_input)

            # Check if the hyperparameters have a valid result or an exception
            validation_result = result_on_validation_list_saved[self.model_counter]

            if validation_result is None:
                # Exception detected
                result_on_validation_list_input.append(+ self.INVALID_CONFIG_VALUE)

                assert self.metadata_dict["exception_list"][self.model_counter] is not None, \
                    "{}: Resuming '{}' Failed due to inconsistent data. Invalid validation result found in position {} but no corresponding exception detected.".format(self.ALGORITHM_NAME, self.output_file_name_root, self.model_counter)
            else:
                result_on_validation_list_input.append(- validation_result[self.metric_to_optimize])



            self.model_counter += 1


        self._print("{}: Resuming '{}'... Loaded {} configurations.".format(self.ALGORITHM_NAME, self.output_file_name_root, self.model_counter))


        # If the data structure exists but is empty, return None
        if len(hyperparameters_list_input) == 0:
            self.resume_from_saved = False
            return None, None

        # If loaded less configurations than desired ones
        if self.model_counter < self.n_calls:
            self.resume_from_saved = False


        return hyperparameters_list_input, result_on_validation_list_input


    def search(self, recommender_input_args,
               parameter_search_space,
               metric_to_optimize = "MAP",
               n_cases = 20,
               n_random_starts = 5,
               output_folder_path = None,
               output_file_name_root = None,
               save_model = "best",
               save_metadata = True,
               resume_from_saved = False,
               recommender_input_args_last_test = None,
               evaluate_on_test_each_best_solution = True,
               ):
        """
        :param recommender_input_args:
        :param parameter_search_space:
        :param metric_to_optimize:
        :param n_cases:
        :param n_random_starts:
        :param output_folder_path:
        :param output_file_name_root:
        :param save_model:          "no"    don't save anything
                                    "all"   save every model
                                    "best"  save the best model trained on train data alone and on last, if present
                                    "last"  save only last, if present
        :param save_metadata:
        :param recommender_input_args_last_test:
        :return:
        """


        self._set_skopt_params()    ### default parameters are set here

        self._set_search_attributes(recommender_input_args,
                                    recommender_input_args_last_test,
                                    metric_to_optimize,
                                    output_folder_path,
                                    output_file_name_root,
                                    resume_from_saved,
                                    save_metadata,
                                    save_model,
                                    evaluate_on_test_each_best_solution,
                                    n_cases)


        self.parameter_search_space = parameter_search_space
        self.n_random_starts = n_random_starts
        self.n_calls = n_cases
        self.n_jobs = 1
        self.n_loaded_counter = 0


        self.hyperparams = dict()
        self.hyperparams_names = list()
        self.hyperparams_values = list()

        skopt_types = [Real, Integer, Categorical]

        for name, hyperparam in self.parameter_search_space.items():

            if any(isinstance(hyperparam, sko_type) for sko_type in skopt_types):
                self.hyperparams_names.append(name)
                self.hyperparams_values.append(hyperparam)
                self.hyperparams[name] = hyperparam

            else:
                raise ValueError("{}: Unexpected parameter type: {} - {}".format(self.ALGORITHM_NAME, str(name), str(hyperparam)))


        if self.resume_from_saved:
            hyperparameters_list_input, result_on_validation_list_saved = self._resume_from_saved()
            self.x0 = hyperparameters_list_input
            self.y0 = result_on_validation_list_saved

            self.n_random_starts = max(0, self.n_random_starts - self.model_counter)
            self.n_calls = max(0, self.n_calls - self.model_counter)
            self.n_loaded_counter = self.model_counter



        self.result = gp_minimize(self._objective_function_list_input,
                                  self.hyperparams_values,
                                  base_estimator=None,
                                  n_calls=self.n_calls,
                                  n_random_starts=self.n_random_starts,
                                  acq_func=self.acq_func,
                                  acq_optimizer=self.acq_optimizer,
                                  x0=self.x0,
                                  y0=self.y0,
                                  random_state=self.random_state,
                                  verbose=self.verbose,
                                  callback=None,
                                  n_points=self.n_point,
                                  n_restarts_optimizer=self.n_restarts_optimizer,
                                  xi=self.xi,
                                  kappa=self.kappa,
                                  noise=self.noise,
                                  n_jobs=self.n_jobs)


        if self.n_loaded_counter < self.model_counter:
            self._write_log("{}: Search complete. Best config is {}: {}\n".format(self.ALGORITHM_NAME,
                                                                           self.metadata_dict["hyperparameters_best_index"],
                                                                           self.metadata_dict["hyperparameters_best"]))


        if self.recommender_input_args_last_test is not None:
            self._evaluate_on_test_with_data_last()







    def _objective_function_list_input(self, current_fit_parameters_list_of_values):

        current_fit_parameters_dict = dict(zip(self.hyperparams_names, current_fit_parameters_list_of_values))

        return self._objective_function(current_fit_parameters_dict)


/opt/conda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [8]:
# hyperparameter_search.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 22/11/17
@author: Maurizio Ferrari Dacrema
"""

# Automate hyperparameter tuning
# using bayesian optimization with scikit-optimize
# -------------------------------------------------

import os, multiprocessing
from functools import partial

######################################################################
from skopt.space import Real, Integer, Categorical
import traceback
# from Utils.PoolWithSubprocess import PoolWithSubprocess


def run_KNNRecommender_on_similarity_type(similarity_type, parameterSearch,
                                          parameter_search_space,
                                          recommender_input_args,
                                          n_cases,
                                          n_random_starts,
                                          resume_from_saved,
                                          save_model,
                                          output_folder_path,
                                          output_file_name_root,
                                          metric_to_optimize,
                                          allow_weighting=False,
                                          recommender_input_args_last_test=None):

    original_parameter_search_space = parameter_search_space

    hyperparameters_range_dictionary = {}
    hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
    hyperparameters_range_dictionary["shrink"] = Integer(0, 1000)
    hyperparameters_range_dictionary["similarity"] = Categorical([similarity_type])
    hyperparameters_range_dictionary["normalize"] = Categorical([True, False])

    is_set_similarity = similarity_type in ["tversky", "dice", "jaccard", "tanimoto"]

    if similarity_type == "asymmetric":
        hyperparameters_range_dictionary["asymmetric_alpha"] = Real(low=0, high=2, prior='uniform')
        hyperparameters_range_dictionary["normalize"] = Categorical([True])

    elif similarity_type == "tversky":
        hyperparameters_range_dictionary["tversky_alpha"] = Real(low=0, high=2, prior='uniform')
        hyperparameters_range_dictionary["tversky_beta"] = Real(low=0, high=2, prior='uniform')
        hyperparameters_range_dictionary["normalize"] = Categorical([True])

    elif similarity_type == "euclidean":
        hyperparameters_range_dictionary["normalize"] = Categorical([True, False])
        hyperparameters_range_dictionary["normalize_avg_row"] = Categorical([True, False])
        hyperparameters_range_dictionary["similarity_from_distance_mode"] = Categorical(["lin", "log", "exp"])

    if not is_set_similarity:

        if allow_weighting:
            hyperparameters_range_dictionary["feature_weighting"] = Categorical(["none", "BM25", "TF-IDF"])

    local_parameter_search_space = {**hyperparameters_range_dictionary, **original_parameter_search_space}

    parameterSearch.search(recommender_input_args,
                           parameter_search_space=local_parameter_search_space,
                           n_cases=n_cases,
                           n_random_starts=n_random_starts,
                           resume_from_saved=resume_from_saved,
                           save_model=save_model,
                           output_folder_path=output_folder_path,
                           output_file_name_root=output_file_name_root + "_" + similarity_type,
                           metric_to_optimize=metric_to_optimize,
                           recommender_input_args_last_test=recommender_input_args_last_test)


def runParameterSearch_Content(recommender_class, URM_train, ICM_object, ICM_name, URM_train_last_test=None,
                               n_cases=30, n_random_starts=5, resume_from_saved=False, save_model="best",
                               evaluator_validation=None, evaluator_test=None, metric_to_optimize="PRECISION",
                               output_folder_path="result_experiments/", parallelizeKNN=False, allow_weighting=True,
                               similarity_type_list=None):
    # If directory does not exist, create
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    URM_train = URM_train.copy()
    ICM_object = ICM_object.copy()

    if URM_train_last_test is not None:
        URM_train_last_test = URM_train_last_test.copy()

    ##########################################################################################################

    output_file_name_root = recommender_class.RECOMMENDER_NAME + "_{}".format(ICM_name)

    parameterSearch = SearchBayesianSkopt(recommender_class,
                                          evaluator_validation=evaluator_validation,
                                          evaluator_test=evaluator_test)

    if similarity_type_list is None:
        similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"]

    recommender_input_args = SearchInputRecommenderArgs(
        CONSTRUCTOR_POSITIONAL_ARGS=[URM_train, ICM_object],
        CONSTRUCTOR_KEYWORD_ARGS={},
        FIT_POSITIONAL_ARGS=[],
        FIT_KEYWORD_ARGS={}
    )

    if URM_train_last_test is not None:
        recommender_input_args_last_test = recommender_input_args.copy()
        recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train_last_test
    else:
        recommender_input_args_last_test = None

    run_KNNCBFRecommender_on_similarity_type_partial = partial(run_KNNRecommender_on_similarity_type,
                                                               recommender_input_args=recommender_input_args,
                                                               parameter_search_space={},
                                                               parameterSearch=parameterSearch,
                                                               n_cases=n_cases,
                                                               n_random_starts=n_random_starts,
                                                               resume_from_saved=resume_from_saved,
                                                               save_model=save_model,
                                                               output_folder_path=output_folder_path,
                                                               output_file_name_root=output_file_name_root,
                                                               metric_to_optimize=metric_to_optimize,
                                                               allow_weighting=allow_weighting,
                                                               recommender_input_args_last_test=recommender_input_args_last_test)

    # if parallelizeKNN:
    #     pool = multiprocessing.Pool(processes=int(multiprocessing.cpu_count()), maxtasksperchild=1)
    #     pool.map(run_KNNCBFRecommender_on_similarity_type_partial, similarity_type_list)
    #
    #     pool.close()
    #     pool.join()
    #
    # else:
    #
    for similarity_type in similarity_type_list:
        run_KNNCBFRecommender_on_similarity_type_partial(similarity_type)


def runParameterSearch_Collaborative(recommender_class, URM_train, URM_train_last_test=None,
                                     metric_to_optimize="PRECISION",
                                     evaluator_validation=None, evaluator_test=None,
                                     evaluator_validation_earlystopping=None,
                                     output_folder_path="result_experiments/", parallelizeKNN=True,
                                     n_cases=35, n_random_starts=5, resume_from_saved=False, save_model="best",
                                     allow_weighting=True,
                                     similarity_type_list=None):

    # If directory does not exist, create
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    earlystopping_keywargs = {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation_earlystopping,
                              "lower_validations_allowed": 5,
                              "validation_metric": metric_to_optimize,
                              }

    URM_train = URM_train.copy()

    if URM_train_last_test is not None:
        URM_train_last_test = URM_train_last_test.copy()

    try:

        output_file_name_root = recommender_class.RECOMMENDER_NAME

        parameterSearch = SearchBayesianSkopt(recommender_class,
                                              evaluator_validation=evaluator_validation,
                                              evaluator_test=evaluator_test)

        # if recommender_class in [TopPop, GlobalEffects, Random]:
        #     """
        #     TopPop, GlobalEffects and Random have no parameters therefore only one evaluation is needed
        #     """
        #
        #     parameterSearch = SearchSingleCase(recommender_class,
        #                                        evaluator_validation=evaluator_validation,
        #                                        evaluator_test=evaluator_test)
        #
        #     recommender_input_args = SearchInputRecommenderArgs(
        #         CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
        #         CONSTRUCTOR_KEYWORD_ARGS={},
        #         FIT_POSITIONAL_ARGS=[],
        #         FIT_KEYWORD_ARGS={}
        #     )
        #
        #     if URM_train_last_test is not None:
        #         recommender_input_args_last_test = recommender_input_args.copy()
        #         recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train_last_test
        #     else:
        #         recommender_input_args_last_test = None
        #
        #     parameterSearch.search(recommender_input_args,
        #                            recommender_input_args_last_test=recommender_input_args_last_test,
        #                            fit_hyperparameters_values={},
        #                            output_folder_path=output_folder_path,
        #                            output_file_name_root=output_file_name_root,
        #                            resume_from_saved=resume_from_saved,
        #                            save_model=save_model,
        #                            )
        #
        #     return

        ##########################################################################################################

        if recommender_class in [ItemKNNCFRecommender, UserKNNCFRecommender]:

            if similarity_type_list is None:
                similarity_type_list = ['cosine', 'jaccard', "asymmetric", "dice", "tversky"]

            recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
                CONSTRUCTOR_KEYWORD_ARGS={},
                FIT_POSITIONAL_ARGS=[],
                FIT_KEYWORD_ARGS={}
            )

            if URM_train_last_test is not None:
                recommender_input_args_last_test = recommender_input_args.copy()
                recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train_last_test
            else:
                recommender_input_args_last_test = None

            run_KNNCFRecommender_on_similarity_type_partial = partial(run_KNNRecommender_on_similarity_type,
                                                                      recommender_input_args=recommender_input_args,
                                                                      parameter_search_space={},
                                                                      parameterSearch=parameterSearch,
                                                                      n_cases=n_cases,
                                                                      n_random_starts=n_random_starts,
                                                                      resume_from_saved=resume_from_saved,
                                                                      save_model=save_model,
                                                                      output_folder_path=output_folder_path,
                                                                      output_file_name_root=output_file_name_root,
                                                                      metric_to_optimize=metric_to_optimize,
                                                                      allow_weighting=allow_weighting,
                                                                      recommender_input_args_last_test=recommender_input_args_last_test)

            # if parallelizeKNN:
            #     pool = multiprocessing.Pool(processes=multiprocessing.cpu_count(), maxtasksperchild=1)
            #     pool.map(run_KNNCFRecommender_on_similarity_type_partial, similarity_type_list)
            #
            #     pool.close()
            #     pool.join()
            #
            # else:

            for similarity_type in similarity_type_list:
                run_KNNCFRecommender_on_similarity_type_partial(similarity_type)

            return

        ##########################################################################################################

        # if recommender_class is P3alphaRecommender:
        #     hyperparameters_range_dictionary = {}
        #     hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
        #     hyperparameters_range_dictionary["alpha"] = Real(low=0, high=2, prior='uniform')
        #     hyperparameters_range_dictionary["normalize_similarity"] = Categorical([True, False])
        #
        #     recommender_input_args = SearchInputRecommenderArgs(
        #         CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
        #         CONSTRUCTOR_KEYWORD_ARGS={},
        #         FIT_POSITIONAL_ARGS=[],
        #         FIT_KEYWORD_ARGS={}
        #     )

        ##########################################################################################################

        # if recommender_class is RP3betaRecommender:
        #     hyperparameters_range_dictionary = {}
        #     hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
        #     hyperparameters_range_dictionary["alpha"] = Real(low=0, high=2, prior='uniform')
        #     hyperparameters_range_dictionary["beta"] = Real(low=0, high=2, prior='uniform')
        #     hyperparameters_range_dictionary["normalize_similarity"] = Categorical([True, False])
        #
        #     recommender_input_args = SearchInputRecommenderArgs(
        #         CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
        #         CONSTRUCTOR_KEYWORD_ARGS={},
        #         FIT_POSITIONAL_ARGS=[],
        #         FIT_KEYWORD_ARGS={}
        #     )

        ##########################################################################################################

        # if recommender_class is MatrixFactorization_FunkSVD_Cython:
        #     hyperparameters_range_dictionary = {}
        #     hyperparameters_range_dictionary["sgd_mode"] = Categorical(["sgd", "adagrad", "adam"])
        #     hyperparameters_range_dictionary["epochs"] = Categorical([500])
        #     hyperparameters_range_dictionary["use_bias"] = Categorical([True, False])
        #     hyperparameters_range_dictionary["batch_size"] = Categorical([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
        #     hyperparameters_range_dictionary["num_factors"] = Integer(1, 200)
        #     hyperparameters_range_dictionary["item_reg"] = Real(low=1e-5, high=1e-2, prior='log-uniform')
        #     hyperparameters_range_dictionary["user_reg"] = Real(low=1e-5, high=1e-2, prior='log-uniform')
        #     hyperparameters_range_dictionary["learning_rate"] = Real(low=1e-4, high=1e-1, prior='log-uniform')
        #     hyperparameters_range_dictionary["negative_interactions_quota"] = Real(low=0.0, high=0.5, prior='uniform')
        #
        #     recommender_input_args = SearchInputRecommenderArgs(
        #         CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
        #         CONSTRUCTOR_KEYWORD_ARGS={},
        #         FIT_POSITIONAL_ARGS=[],
        #         FIT_KEYWORD_ARGS=earlystopping_keywargs
        #     )

        ##########################################################################################################

        # if recommender_class is MatrixFactorization_AsySVD_Cython:
        #     hyperparameters_range_dictionary = {}
        #     hyperparameters_range_dictionary["sgd_mode"] = Categorical(["sgd", "adagrad", "adam"])
        #     hyperparameters_range_dictionary["epochs"] = Categorical([500])
        #     hyperparameters_range_dictionary["use_bias"] = Categorical([True, False])
        #     hyperparameters_range_dictionary["batch_size"] = Categorical([1])
        #     hyperparameters_range_dictionary["num_factors"] = Integer(1, 200)
        #     hyperparameters_range_dictionary["item_reg"] = Real(low=1e-5, high=1e-2, prior='log-uniform')
        #     hyperparameters_range_dictionary["user_reg"] = Real(low=1e-5, high=1e-2, prior='log-uniform')
        #     hyperparameters_range_dictionary["learning_rate"] = Real(low=1e-4, high=1e-1, prior='log-uniform')
        #     hyperparameters_range_dictionary["negative_interactions_quota"] = Real(low=0.0, high=0.5, prior='uniform')
        #
        #     recommender_input_args = SearchInputRecommenderArgs(
        #         CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
        #         CONSTRUCTOR_KEYWORD_ARGS={},
        #         FIT_POSITIONAL_ARGS=[],
        #         FIT_KEYWORD_ARGS=earlystopping_keywargs
        #     )

        ##########################################################################################################

#         if recommender_class is MatrixFactorization_BPR_Cython:
#             hyperparameters_range_dictionary = {}
#             hyperparameters_range_dictionary["sgd_mode"] = Categorical(["sgd", "adagrad", "adam"])
#             hyperparameters_range_dictionary["epochs"] = Categorical([1500]) 
#             hyperparameters_range_dictionary["num_factors"] = Integer(1, 200)
#             hyperparameters_range_dictionary["batch_size"] = Categorical([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
#             hyperparameters_range_dictionary["positive_reg"] = Real(low=1e-5, high=1e-2, prior='log-uniform')
#             hyperparameters_range_dictionary["negative_reg"] = Real(low=1e-5, high=1e-2, prior='log-uniform')
#             hyperparameters_range_dictionary["learning_rate"] = Real(low=1e-4, high=1e-1, prior='log-uniform')
        
#             recommender_input_args = SearchInputRecommenderArgs(
#                 CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
#                 CONSTRUCTOR_KEYWORD_ARGS={},
#                 FIT_POSITIONAL_ARGS=[],
#                 FIT_KEYWORD_ARGS={**earlystopping_keywargs,
#                                   "positive_threshold_BPR": None}
#             )

        ##########################################################################################################

        # if recommender_class is IALSRecommender:
        #     hyperparameters_range_dictionary = {}
        #     hyperparameters_range_dictionary["num_factors"] = Integer(1, 200)
        #     hyperparameters_range_dictionary["confidence_scaling"] = Categorical(["linear", "log"])
        #     hyperparameters_range_dictionary["alpha"] = Real(low=1e-3, high=50.0, prior='log-uniform')
        #     hyperparameters_range_dictionary["epsilon"] = Real(low=1e-3, high=10.0, prior='log-uniform')
        #     hyperparameters_range_dictionary["reg"] = Real(low=1e-5, high=1e-2, prior='log-uniform')
        #
        #     recommender_input_args = SearchInputRecommenderArgs(
        #         CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
        #         CONSTRUCTOR_KEYWORD_ARGS={},
        #         FIT_POSITIONAL_ARGS=[],
        #         FIT_KEYWORD_ARGS=earlystopping_keywargs
        #     )

        ##########################################################################################################

        if recommender_class is PureSVDRecommender:
            hyperparameters_range_dictionary = {}
            hyperparameters_range_dictionary["num_factors"] = Integer(1, 350)

            recommender_input_args = SearchInputRecommenderArgs(
                CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
                CONSTRUCTOR_KEYWORD_ARGS={},
                FIT_POSITIONAL_ARGS=[],
                FIT_KEYWORD_ARGS={}
            )

        ##########################################################################################################

        # if recommender_class is NMFRecommender:
        #     hyperparameters_range_dictionary = {}
        #     hyperparameters_range_dictionary["num_factors"] = Integer(1, 350)
        #     hyperparameters_range_dictionary["solver"] = Categorical(["coordinate_descent", "multiplicative_update"])
        #     hyperparameters_range_dictionary["init_type"] = Categorical(["random", "nndsvda"])
        #     hyperparameters_range_dictionary["beta_loss"] = Categorical(["frobenius", "kullback-leibler"])
        #
        #     recommender_input_args = SearchInputRecommenderArgs(
        #         CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
        #         CONSTRUCTOR_KEYWORD_ARGS={},
        #         FIT_POSITIONAL_ARGS=[],
        #         FIT_KEYWORD_ARGS={}
        #     )

        #########################################################################################################

        # if recommender_class is SLIM_BPR_Cython:
        #     hyperparameters_range_dictionary = {}
        #     hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
        #     hyperparameters_range_dictionary["epochs"] = Categorical([1500])
        #     hyperparameters_range_dictionary["symmetric"] = Categorical([True, False])
        #     hyperparameters_range_dictionary["sgd_mode"] = Categorical(["sgd", "adagrad", "adam"])
        #     hyperparameters_range_dictionary["lambda_i"] = Real(low=1e-5, high=1e-2, prior='log-uniform')
        #     hyperparameters_range_dictionary["lambda_j"] = Real(low=1e-5, high=1e-2, prior='log-uniform')
        #     hyperparameters_range_dictionary["learning_rate"] = Real(low=1e-4, high=1e-1, prior='log-uniform')
        #
        #     recommender_input_args = SearchInputRecommenderArgs(
        #         CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
        #         CONSTRUCTOR_KEYWORD_ARGS={},
        #         FIT_POSITIONAL_ARGS=[],
        #         FIT_KEYWORD_ARGS={**earlystopping_keywargs,
        #                           "positive_threshold_BPR": None,
        #                           'train_with_sparse_weights': None}
        #     )

        ##########################################################################################################
        #
        # if recommender_class is SLIMElasticNetRecommender:
        #     hyperparameters_range_dictionary = {}
        #     hyperparameters_range_dictionary["topK"] = Integer(5, 1000)
        #     hyperparameters_range_dictionary["l1_ratio"] = Real(low=1e-5, high=1.0, prior='log-uniform')
        #     hyperparameters_range_dictionary["alpha"] = Real(low=1e-3, high=1.0, prior='uniform')
        #
        #     recommender_input_args = SearchInputRecommenderArgs(
        #         CONSTRUCTOR_POSITIONAL_ARGS=[URM_train],
        #         CONSTRUCTOR_KEYWORD_ARGS={},
        #         FIT_POSITIONAL_ARGS=[],
        #         FIT_KEYWORD_ARGS={}
        #     )

        #########################################################################################################

        if URM_train_last_test is not None:
            recommender_input_args_last_test = recommender_input_args.copy()
            recommender_input_args_last_test.CONSTRUCTOR_POSITIONAL_ARGS[0] = URM_train_last_test
        else:
            recommender_input_args_last_test = None

        ## Final step, after the hyperparameter range has been defined for each type of algorithm
        parameterSearch.search(recommender_input_args,
                               parameter_search_space=hyperparameters_range_dictionary,
                               n_cases=n_cases,
                               n_random_starts=n_random_starts,
                               resume_from_saved=resume_from_saved,
                               save_model=save_model,
                               output_folder_path=output_folder_path,
                               output_file_name_root=output_file_name_root,
                               metric_to_optimize=metric_to_optimize,
                               recommender_input_args_last_test=recommender_input_args_last_test)

    except Exception as e:

        print("On recommender {} Exception {}".format(recommender_class, str(e)))
        traceback.print_exc()

        error_file = open(output_folder_path + "ErrorLog.txt", "a")
        error_file.write("On recommender {} Exception {}\n".format(recommender_class, str(e)))
        error_file.close()


> **Data IO**

In [9]:
# DataIO.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 27/04/2019
@author: Maurizio Ferrari Dacrema
"""

import os, json, zipfile, shutil, platform

import scipy.sparse as sps
from pandas import DataFrame
import pandas as pd
import numpy as np


def json_not_serializable_handler(o):
    """
    Json cannot serialize automatically some data types, for example numpy integers (int32).
    This may be a limitation of numpy-json interfaces for Python 3.6 and may not occur in Python 3.7
    :param o:
    :return:
    """

    if isinstance(o, np.integer):
        return int(o)

    raise TypeError("json_not_serializable_handler: object '{}' is not serializable.".format(type(o)))


class DataIO(object):
    """ DataIO"""

    _DEFAULT_TEMP_FOLDER = ".temp_DataIO_"

    # _MAX_PATH_LENGTH_LINUX = 4096
    _MAX_PATH_LENGTH_WINDOWS = 255

    def __init__(self, folder_path):
        super(DataIO, self).__init__()

        self._is_windows = platform.system() == "Windows"

        self.folder_path = folder_path
        self._key_string_alert_done = False

        # if self._is_windows:
        #     self.folder_path = "\\\\?\\" + self.folder_path


    def _print(self, message):
        print("{}: {}".format("DataIO", message))


    def _get_temp_folder(self, file_name):
        """
        Creates a temporary folder to be used during the data saving
        :return:
        """

        # Ignore the .zip extension
        file_name = file_name[:-4]

        current_temp_folder = "{}{}_{}/".format(self.folder_path, self._DEFAULT_TEMP_FOLDER, file_name)

        if os.path.exists(current_temp_folder):
            self._print("Folder {} already exists, could be the result of a previous failed save attempt or multiple saver are active in parallel. " \
            "Folder will be removed.".format(current_temp_folder))

            shutil.rmtree(current_temp_folder, ignore_errors=True)

        os.makedirs(current_temp_folder)

        return current_temp_folder


    def _check_dict_key_type(self, dict_to_save):
        """
        Check whether the keys of the dictionary are string. If not, transforms them into strings
        :param dict_to_save:
        :return:
        """

        all_keys_are_str = all(isinstance(key, str) for key in dict_to_save.keys())

        if all_keys_are_str:
            return dict_to_save

        if not self._key_string_alert_done:
            self._print("Json dumps supports only 'str' as dictionary keys. Transforming keys to string, note that this will alter the mapper content.")
            self._key_string_alert_done = True

        dict_to_save_key_str = {str(key):val for (key,val) in dict_to_save.items()}

        assert all(dict_to_save_key_str[str(key)] == val for (key,val) in dict_to_save.items()), \
            "DataIO: Transforming dictionary keys into strings altered its content. Duplicate keys may have been produced."

        return dict_to_save_key_str


    def save_data(self, file_name, data_dict_to_save):

        # If directory does not exist, create with .temp_model_folder
        if not os.path.exists(self.folder_path):
            os.makedirs(self.folder_path)

        if file_name[-4:] != ".zip":
            file_name += ".zip"


        current_temp_folder = self._get_temp_folder(file_name)

        attribute_to_file_name = {}
        attribute_to_json_file = {}

        for attrib_name, attrib_data in data_dict_to_save.items():

            current_file_path = current_temp_folder + attrib_name

            if isinstance(attrib_data, DataFrame):
                attrib_data.to_csv(current_file_path, index=False)
                attribute_to_file_name[attrib_name] = attrib_name + ".csv"

            elif isinstance(attrib_data, sps.spmatrix):
                sps.save_npz(current_file_path, attrib_data)
                attribute_to_file_name[attrib_name] = attrib_name + ".npz"

            elif isinstance(attrib_data, np.ndarray):
                # allow_pickle is FALSE to prevent using pickle and ensure portability
                np.save(current_file_path, attrib_data, allow_pickle=False)
                attribute_to_file_name[attrib_name] = attrib_name + ".npy"

            else:
                attribute_to_json_file[attrib_name] = attrib_data
                attribute_to_file_name[attrib_name] = attrib_name + ".json"


        # Save list objects
        attribute_to_json_file[".DataIO_attribute_to_file_name"] = attribute_to_file_name.copy()

        for attrib_name, attrib_data in attribute_to_json_file.items():

            current_file_path = current_temp_folder + attrib_name
            attribute_to_file_name[attrib_name] = attrib_name + ".json"

            # if self._is_windows and len(current_file_path + ".json") >= self._MAX_PATH_LENGTH_WINDOWS:
            #     current_file_path = "\\\\?\\" + current_file_path

            absolute_path = current_file_path + ".json" if current_file_path.startswith(os.getcwd()) else os.getcwd() + current_file_path + ".json"

            assert not self._is_windows or (self._is_windows and len(absolute_path) <= self._MAX_PATH_LENGTH_WINDOWS), \
                "DataIO: Path of file exceeds {} characters, which is the maximum allowed under standard paths for Windows.".format(self._MAX_PATH_LENGTH_WINDOWS)


            with open(current_file_path + ".json", 'w') as outfile:

                if isinstance(attrib_data, dict):
                    attrib_data = self._check_dict_key_type(attrib_data)

                json.dump(attrib_data, outfile, default=json_not_serializable_handler)



        with zipfile.ZipFile(self.folder_path + file_name, 'w', compression=zipfile.ZIP_DEFLATED) as myzip:

            for file_name in attribute_to_file_name.values():
                myzip.write(current_temp_folder + file_name, arcname = file_name)



        shutil.rmtree(current_temp_folder, ignore_errors=True)


    def load_data(self, file_name):

        if file_name[-4:] != ".zip":
            file_name += ".zip"

        dataFile = zipfile.ZipFile(self.folder_path + file_name)

        dataFile.testzip()

        current_temp_folder = self._get_temp_folder(file_name)

        try:

            try:
                attribute_to_file_name_path = dataFile.extract(".DataIO_attribute_to_file_name.json", path = current_temp_folder)
            except KeyError:
                attribute_to_file_name_path = dataFile.extract("__DataIO_attribute_to_file_name.json", path = current_temp_folder)


            with open(attribute_to_file_name_path, "r") as json_file:
                attribute_to_file_name = json.load(json_file)

            data_dict_loaded = {}

            for attrib_name, file_name in attribute_to_file_name.items():

                attrib_file_path = dataFile.extract(file_name, path = current_temp_folder)
                attrib_data_type = file_name.split(".")[-1]

                if attrib_data_type == "csv":
                    attrib_data = pd.read_csv(attrib_file_path, index_col=False)

                elif attrib_data_type == "npz":
                    attrib_data = sps.load_npz(attrib_file_path)

                elif attrib_data_type == "npy":
                    # allow_pickle is FALSE to prevent using pickle and ensure portability
                    attrib_data = np.load(attrib_file_path, allow_pickle=False)

                elif attrib_data_type == "json":
                    with open(attrib_file_path, "r") as json_file:
                        attrib_data = json.load(json_file)

                else:
                    raise Exception("Attribute type not recognized for: '{}' of class: '{}'".format(attrib_file_path, attrib_data_type))

                data_dict_loaded[attrib_name] = attrib_data


        except Exception as exec:

            shutil.rmtree(current_temp_folder, ignore_errors=True)
            raise exec

        shutil.rmtree(current_temp_folder, ignore_errors=True)


        return data_dict_loaded

> **Create submission file**

In [10]:
# create_submission_file.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
#  -*- coding: utf-8 -*-

"""
	create_submission_file.py: module for creating the formatted csv submission file.
"""

import os
from datetime import datetime


def create_csv(top_10_items, recommender):
    print("\nGenerating submission csv ... ")

    # save on a different dir according to the recommender used
    submissions_dir = './submissions/' + recommender

    # If directory for the recommender does not exist, create
    if not os.path.exists(submissions_dir):
        os.makedirs(submissions_dir)

    csv_fname = 'submission_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    file_path = os.path.join(submissions_dir, csv_fname)

    with open(file_path, 'w') as f:

        fieldnames = 'user_id,item_list'
        f.write(fieldnames + '\n')

        for user_id, item_list in top_10_items.items():
            row = str(user_id) + ',' + str(item_list) + '\n'
            f.write(row.replace('[', '').replace(']', ''))  # remove '[' ']' from item_list string


> **Compute similarity**

In [11]:
# compute_similarity.py
# ------------------------------------------------------------------


#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 23/10/17
@author: Maurizio Ferrari Dacrema
"""

import numpy as np
import time, sys
import scipy.sparse as sps
# from utils import data_manager as dp


class Compute_Similarity_Python:

    def __init__(self, dataMatrix, topK=100, shrink=0, normalize=True,
                 asymmetric_alpha=0.5, tversky_alpha=1.0, tversky_beta=1.0,
                 similarity="cosine", row_weights=None):
        """
        Computes the cosine similarity on the columns of dataMatrix
        If it is computed on URM=|users|x|items|, pass the URM as is.
        If it is computed on ICM=|items|x|features|, pass the ICM transposed.
        :param dataMatrix:
        :param topK:
        :param shrink:
        :param normalize:           If True divide the dot product by the product of the norms
        :param row_weights:         Multiply the values in each row by a specified value. Array
        :param asymmetric_alpha     Coefficient alpha for the asymmetric cosine
        :param similarity:  "cosine"        computes Cosine similarity
                            "adjusted"      computes Adjusted Cosine, removing the average of the users
                            "asymmetric"    computes Asymmetric Cosine
                            "pearson"       computes Pearson Correlation, removing the average of the items
                            "jaccard"       computes Jaccard similarity for binary interactions using Tanimoto
                            "dice"          computes Dice similarity for binary interactions
                            "tversky"       computes Tversky similarity for binary interactions
                            "tanimoto"      computes Tanimoto coefficient for binary interactions
        """

        super(Compute_Similarity_Python, self).__init__()

        self.shrink = shrink
        self.normalize = normalize

        self.n_rows, self.n_columns = dataMatrix.shape
        self.TopK = min(topK, self.n_columns)

        self.asymmetric_alpha = asymmetric_alpha
        self.tversky_alpha = tversky_alpha
        self.tversky_beta = tversky_beta

        self.dataMatrix = dataMatrix.copy()

        self.adjusted_cosine = False
        self.asymmetric_cosine = False
        self.pearson_correlation = False
        self.tanimoto_coefficient = False
        self.dice_coefficient = False
        self.tversky_coefficient = False

        if similarity == "adjusted":
            self.adjusted_cosine = True
        elif similarity == "asymmetric":
            self.asymmetric_cosine = True
        elif similarity == "pearson":
            self.pearson_correlation = True
        elif similarity == "jaccard" or similarity == "tanimoto":
            self.tanimoto_coefficient = True
            # Tanimoto has a specific kind of normalization
            self.normalize = False

        elif similarity == "dice":
            self.dice_coefficient = True
            self.normalize = False

        elif similarity == "tversky":
            self.tversky_coefficient = True
            self.normalize = False

        elif similarity == "cosine":
            pass
        else:
            raise ValueError("Cosine_Similarity: value for parameter 'mode' not recognized."
                             " Allowed values are: 'cosine', 'pearson', 'adjusted', 'asymmetric', 'jaccard', 'tanimoto',"
                             "dice, tversky."
                             " Passed value was '{}'".format(similarity))

        self.use_row_weights = False

        if row_weights is not None:

            if dataMatrix.shape[0] != len(row_weights):
                raise ValueError("Cosine_Similarity: provided row_weights and dataMatrix have different number of rows."
                                 "Col_weights has {} columns, dataMatrix has {}.".format(len(row_weights),
                                                                                         dataMatrix.shape[0]))

            self.use_row_weights = True
            self.row_weights = row_weights.copy()
            self.row_weights_diag = sps.diags(self.row_weights)

            self.dataMatrix_weighted = self.dataMatrix.T.dot(self.row_weights_diag).T

    def applyAdjustedCosine(self):
        """
        Remove from every data point the average for the corresponding row
        :return:
        """

        self.dataMatrix = check_matrix(self.dataMatrix, 'csr')

        interactionsPerRow = np.diff(self.dataMatrix.indptr)

        nonzeroRows = interactionsPerRow > 0
        sumPerRow = np.asarray(self.dataMatrix.sum(axis=1)).ravel()

        rowAverage = np.zeros_like(sumPerRow)
        rowAverage[nonzeroRows] = sumPerRow[nonzeroRows] / interactionsPerRow[nonzeroRows]

        # Split in blocks to avoid duplicating the whole data structure
        start_row = 0
        end_row = 0

        blockSize = 1000

        while end_row < self.n_rows:
            end_row = min(self.n_rows, end_row + blockSize)

            self.dataMatrix.data[self.dataMatrix.indptr[start_row]:self.dataMatrix.indptr[end_row]] -= \
                np.repeat(rowAverage[start_row:end_row], interactionsPerRow[start_row:end_row])

            start_row += blockSize

    def applyPearsonCorrelation(self):
        """
        Remove from every data point the average for the corresponding column
        :return:
        """

        self.dataMatrix = check_matrix(self.dataMatrix, 'csc')

        interactionsPerCol = np.diff(self.dataMatrix.indptr)

        nonzeroCols = interactionsPerCol > 0
        sumPerCol = np.asarray(self.dataMatrix.sum(axis=0)).ravel()

        colAverage = np.zeros_like(sumPerCol)
        colAverage[nonzeroCols] = sumPerCol[nonzeroCols] / interactionsPerCol[nonzeroCols]

        # Split in blocks to avoid duplicating the whole data structure
        start_col = 0
        end_col = 0

        blockSize = 1000

        while end_col < self.n_columns:
            end_col = min(self.n_columns, end_col + blockSize)

            self.dataMatrix.data[self.dataMatrix.indptr[start_col]:self.dataMatrix.indptr[end_col]] -= \
                np.repeat(colAverage[start_col:end_col], interactionsPerCol[start_col:end_col])

            start_col += blockSize

    def useOnlyBooleanInteractions(self):

        # Split in blocks to avoid duplicating the whole data structure
        start_pos = 0
        end_pos = 0

        blockSize = 1000

        while end_pos < len(self.dataMatrix.data):
            end_pos = min(len(self.dataMatrix.data), end_pos + blockSize)

            self.dataMatrix.data[start_pos:end_pos] = np.ones(end_pos - start_pos)

            start_pos += blockSize

    def compute_similarity(self, start_col=None, end_col=None, block_size=100):
        """
        Compute the similarity for the given dataset
        :param self:
        :param start_col: column to begin with
        :param end_col: column to stop before, end_col is excluded
        :return:
        """

        values = []
        rows = []
        cols = []

        start_time = time.time()
        start_time_print_batch = start_time
        processedItems = 0

        if self.adjusted_cosine:
            self.applyAdjustedCosine()

        elif self.pearson_correlation:
            self.applyPearsonCorrelation()

        elif self.tanimoto_coefficient or self.dice_coefficient or self.tversky_coefficient:
            self.useOnlyBooleanInteractions()

        # We explore the matrix column-wise
        self.dataMatrix = check_matrix(self.dataMatrix, 'csc')

        # Compute sum of squared values to be used in normalization
        sumOfSquared = np.array(self.dataMatrix.power(2).sum(axis=0)).ravel()

        # Tanimoto does not require the square root to be applied
        if not (self.tanimoto_coefficient or self.dice_coefficient or self.tversky_coefficient):
            sumOfSquared = np.sqrt(sumOfSquared)

        if self.asymmetric_cosine:
            sumOfSquared_to_1_minus_alpha = np.power(sumOfSquared, 2 * (1 - self.asymmetric_alpha))
            sumOfSquared_to_alpha = np.power(sumOfSquared, 2 * self.asymmetric_alpha)

        self.dataMatrix = check_matrix(self.dataMatrix, 'csc')

        start_col_local = 0
        end_col_local = self.n_columns

        if start_col is not None and start_col > 0 and start_col < self.n_columns:
            start_col_local = start_col

        if end_col is not None and end_col > start_col_local and end_col < self.n_columns:
            end_col_local = end_col

        start_col_block = start_col_local

        this_block_size = 0

        # Compute all similarities for each item using vectorization
        while start_col_block < end_col_local:

            end_col_block = min(start_col_block + block_size, end_col_local)
            this_block_size = end_col_block - start_col_block

            # All data points for a given item
            item_data = self.dataMatrix[:, start_col_block:end_col_block]
            item_data = item_data.toarray().squeeze()

            # If only 1 feature avoid last dimension to disappear
            if item_data.ndim == 1:
                item_data = np.atleast_2d(item_data)

            if self.use_row_weights:
                this_block_weights = self.dataMatrix_weighted.T.dot(item_data)

            else:
                # Compute item similarities => using dot product
                this_block_weights = self.dataMatrix.T.dot(item_data)

            for col_index_in_block in range(this_block_size):

                if this_block_size == 1:
                    this_column_weights = this_block_weights
                else:
                    this_column_weights = this_block_weights[:, col_index_in_block]

                columnIndex = col_index_in_block + start_col_block
                this_column_weights[columnIndex] = 0.0

                # Apply normalization and shrinkage, ensure denominator != 0
                if self.normalize:

                    if self.asymmetric_cosine:
                        denominator = sumOfSquared_to_alpha[
                                          columnIndex] * sumOfSquared_to_1_minus_alpha + self.shrink + 1e-6
                    else:
                        denominator = sumOfSquared[columnIndex] * sumOfSquared + self.shrink + 1e-6

                    this_column_weights = np.multiply(this_column_weights, 1 / denominator)


                # Apply the specific denominator for Tanimoto
                elif self.tanimoto_coefficient:
                    denominator = sumOfSquared[columnIndex] + sumOfSquared - this_column_weights + self.shrink + 1e-6
                    this_column_weights = np.multiply(this_column_weights, 1 / denominator)

                elif self.dice_coefficient:
                    denominator = sumOfSquared[columnIndex] + sumOfSquared + self.shrink + 1e-6
                    this_column_weights = np.multiply(this_column_weights, 1 / denominator)

                elif self.tversky_coefficient:
                    denominator = this_column_weights + \
                                  (sumOfSquared[columnIndex] - this_column_weights) * self.tversky_alpha + \
                                  (sumOfSquared - this_column_weights) * self.tversky_beta + self.shrink + 1e-6
                    this_column_weights = np.multiply(this_column_weights, 1 / denominator)

                # If no normalization or tanimoto is selected, apply only shrink
                elif self.shrink != 0:
                    this_column_weights = this_column_weights / self.shrink

                # this_column_weights = this_column_weights.toarray().ravel()

                # Sort indices and select TopK
                # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
                # - Partition the data to extract the set of relevant items
                # - Sort only the relevant items
                # - Get the original item index
                relevant_items_partition = (-this_column_weights).argpartition(self.TopK - 1)[0:self.TopK]
                relevant_items_partition_sorting = np.argsort(-this_column_weights[relevant_items_partition])
                top_k_idx = relevant_items_partition[relevant_items_partition_sorting]

                # Incrementally build sparse matrix, do not add zeros
                notZerosMask = this_column_weights[top_k_idx] != 0.0
                numNotZeros = np.sum(notZerosMask)

                values.extend(this_column_weights[top_k_idx][notZerosMask])
                rows.extend(top_k_idx[notZerosMask])
                cols.extend(np.ones(numNotZeros) * columnIndex)

            # Add previous block size
            processedItems += this_block_size

            if time.time() - start_time_print_batch >= 30 or end_col_block == end_col_local:
                columnPerSec = processedItems / (time.time() - start_time + 1e-9)

                print("Similarity column {} ( {:2.0f} % ), {:.2f} column/sec, elapsed time {:.2f} min".format(
                    processedItems, processedItems / (end_col_local - start_col_local) * 100, columnPerSec,
                                    (time.time() - start_time) / 60))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_print_batch = time.time()

            start_col_block += block_size

        # End while on columns

        W_sparse = sps.csr_matrix((values, (rows, cols)),
                                  shape=(self.n_columns, self.n_columns),
                                  dtype=np.float32)

        return W_sparse


def similarityMatrixTopK(item_weights, forceSparseOutput=True, k=100, verbose=False, inplace=True):
    """
    The function selects the TopK most similar elements, column-wise

    :param item_weights:
    :param forceSparseOutput:
    :param k:
    :param verbose:
    :param inplace: Default True, WARNING matrix will be modified
    :return:
    """

    assert (item_weights.shape[0] == item_weights.shape[1]), "selectTopK: ItemWeights is not a square matrix"

    start_time = time.time()

    if verbose:
        print("Generating topK matrix")

    nitems = item_weights.shape[1]
    k = min(k, nitems)

    # for each column, keep only the top-k scored items
    sparse_weights = not isinstance(item_weights, np.ndarray)

    if not sparse_weights:

        idx_sorted = np.argsort(item_weights, axis=0)  # sort data inside each column

        if inplace:
            W = item_weights
        else:
            W = item_weights.copy()

        # index of the items that don't belong to the top-k similar items of each column
        not_top_k = idx_sorted[:-k, :]
        # use numpy fancy indexing to zero-out the values in sim without using a for loop
        W[not_top_k, np.arange(nitems)] = 0.0

        if forceSparseOutput:
            W_sparse = sps.csr_matrix(W, shape=(nitems, nitems))

            if verbose:
                print("Sparse TopK matrix generated in {:.2f} seconds".format(time.time() - start_time))

            return W_sparse

        if verbose:
            print("Dense TopK matrix generated in {:.2f} seconds".format(time.time() - start_time))

        return W

    else:
        # iterate over each column and keep only the top-k similar items
        data, rows_indices, cols_indptr = [], [], []

        item_weights = check_matrix(item_weights, format='csc', dtype=np.float32)

        for item_idx in range(nitems):
            cols_indptr.append(len(data))

            start_position = item_weights.indptr[item_idx]
            end_position = item_weights.indptr[item_idx + 1]

            column_data = item_weights.data[start_position:end_position]
            column_row_index = item_weights.indices[start_position:end_position]

            non_zero_data = column_data != 0

            idx_sorted = np.argsort(column_data[non_zero_data])  # sort by column
            top_k_idx = idx_sorted[-k:]

            data.extend(column_data[non_zero_data][top_k_idx])
            rows_indices.extend(column_row_index[non_zero_data][top_k_idx])

        cols_indptr.append(len(data))

        # During testing CSR is faster
        W_sparse = sps.csc_matrix((data, rows_indices, cols_indptr), shape=(nitems, nitems), dtype=np.float32)
        W_sparse = W_sparse.tocsr()

        if verbose:
            print("Sparse TopK matrix generated in {:.2f} seconds".format(time.time() - start_time))

        return W_sparse


# Transforms matrix into a specific format
def check_matrix(X, format='csc', dtype=np.float32):
    """
        This function takes a matrix as input and transforms it into the specified format.
        The matrix in input can be either sparse or ndarray.
        If the matrix in input has already the desired format, it is returned as-is
        the dtype parameter is always applied and the default is np.float32
        :param X:
        :param format:
        :param dtype:
        :return:
    """

    if format == 'csc' and not isinstance(X, sps.csc_matrix):
        return X.tocsc().astype(dtype)  # Compressed Sparse Column format
    elif format == 'csr' and not isinstance(X, sps.csr_matrix):
        return X.tocsr().astype(dtype)  # Compressed Sparse Row format
    elif format == 'coo' and not isinstance(X, sps.coo_matrix):
        return X.tocoo().astype(dtype)
    elif format == 'dok' and not isinstance(X, sps.dok_matrix):
        return X.todok().astype(dtype)
    elif format == 'bsr' and not isinstance(X, sps.bsr_matrix):
        return X.tobsr().astype(dtype)
    elif format == 'dia' and not isinstance(X, sps.dia_matrix):
        return X.todia().astype(dtype)
    elif format == 'lil' and not isinstance(X, sps.lil_matrix):
        return X.tolil().astype(dtype)
    elif isinstance(X, np.ndarray):
        X = sps.csr_matrix(X, dtype=dtype)
        X.eliminate_zeros()
        return check_matrix(X, format=format, dtype=dtype)
    else:
        return X.astype(dtype)



from enum import Enum

class SimilarityFunction(Enum):
    COSINE = "cosine"
    PEARSON = "pearson"
    JACCARD = "jaccard"
    TANIMOTO = "tanimoto"
    ADJUSTED_COSINE = "adjusted"
    EUCLIDEAN = "euclidean"


class Compute_Similarity:


    def __init__(self, dataMatrix, use_implementation = "density", similarity = None, **args):
        """
        Interface object that will call the appropriate similarity implementation
        :param dataMatrix:
        :param use_implementation:      "density" will choose the most efficient implementation automatically
                                        "cython" will use the cython implementation, if available. Most efficient for sparse matrix
                                        "python" will use the python implementation. Most efficent for dense matrix
        :param similarity:              the type of similarity to use, see SimilarityFunction enum
        :param args:                    other args required by the specific similarity implementation
        """

        assert np.all(np.isfinite(dataMatrix.data)), \
            "Compute_Similarity: Data matrix contains {} non finite values".format(np.sum(np.logical_not(np.isfinite(dataMatrix.data))))

        self.dense = False

        if similarity == "euclidean":
            # This is only available here
            self.compute_similarity_object = Compute_Similarity_Euclidean(dataMatrix, **args)

        else:

            assert not (dataMatrix.shape[0] == 1 and dataMatrix.nnz == dataMatrix.shape[1]),\
                "Compute_Similarity: data has only 1 feature (shape: {}) with dense values," \
                " vector and set based similarities are not defined on 1-dimensional dense data," \
                " use Euclidean similarity instead.".format(dataMatrix.shape)

            if similarity is not None:
                args["similarity"] = similarity


            if use_implementation == "density":

                if isinstance(dataMatrix, np.ndarray):
                    self.dense = True

                elif isinstance(dataMatrix, sps.spmatrix):
                    shape = dataMatrix.shape

                    num_cells = shape[0]*shape[1]

                    sparsity = dataMatrix.nnz/num_cells

                    self.dense = sparsity > 0.5

                else:
                    print("Compute_Similarity: matrix type not recognized, calling default...")
                    use_implementation = "python"

                if self.dense:
                    print("Compute_Similarity: detected dense matrix")
                    use_implementation = "python"
                else:
                    use_implementation = "cython"


            if use_implementation == "cython":

                try:
                    from Base.Similarity.Cython.Compute_Similarity_Cython import Compute_Similarity_Cython
                    self.compute_similarity_object = Compute_Similarity_Cython(dataMatrix, **args)

                except ImportError:
                    print("Unable to load Cython Compute_Similarity, reverting to Python")
                    self.compute_similarity_object = Compute_Similarity_Python(dataMatrix, **args)


            elif use_implementation == "python":
                self.compute_similarity_object = Compute_Similarity_Python(dataMatrix, **args)

            else:

                raise  ValueError("Compute_Similarity: value for argument 'use_implementation' not recognized")


    def compute_similarity(self,  **args):

        return self.compute_similarity_object.compute_similarity(**args)


class Compute_Similarity_Euclidean:

    def __init__(self, dataMatrix, topK=100, shrink = 0, normalize=False, normalize_avg_row=False,
                 similarity_from_distance_mode ="lin", row_weights = None, **args):
        """
        Computes the euclidean similarity on the columns of dataMatrix
        If it is computed on URM=|users|x|items|, pass the URM as is.
        If it is computed on ICM=|items|x|features|, pass the ICM transposed.
        :param dataMatrix:
        :param topK:
        :param normalize
        :param row_weights:         Multiply the values in each row by a specified value. Array
        :param similarity_from_distance_mode:       "exp"   euclidean_similarity = 1/(e ^ euclidean_distance)
                                                    "lin"        euclidean_similarity = 1/(1 + euclidean_distance)
                                                    "log"        euclidean_similarity = 1/(1 + euclidean_distance)
        :param args:                accepts other parameters not needed by the current object
        """

        super(Compute_Similarity_Euclidean, self).__init__()

        self.shrink = shrink
        self.normalize = normalize
        self.normalize_avg_row = normalize_avg_row

        self.n_rows, self.n_columns = dataMatrix.shape
        self.TopK = min(topK, self.n_columns)

        self.dataMatrix = dataMatrix.copy()

        self.similarity_is_exp = False
        self.similarit_is_lin = False
        self.similarity_is_log = False

        if similarity_from_distance_mode == "exp":
            self.similarity_is_exp = True
        elif similarity_from_distance_mode == "lin":
            self.similarity_is_lin = True
        elif similarity_from_distance_mode == "log":
            self.similarity_is_log = True
        else:
            raise ValueError("Compute_Similarity_Euclidean: value for parameter 'mode' not recognized."
                             " Allowed values are: 'exp', 'lin', 'log'."
                             " Passed value was '{}'".format(similarity_from_distance_mode))

        self.use_row_weights = False

        if row_weights is not None:

            if dataMatrix.shape[0] != len(row_weights):
                raise ValueError("Compute_Similarity_Euclidean: provided row_weights and dataMatrix have different number of rows."
                                 "row_weights has {} rows, dataMatrix has {}.".format(len(row_weights), dataMatrix.shape[0]))

            self.use_row_weights = True
            self.row_weights = row_weights.copy()
            self.row_weights_diag = sps.diags(self.row_weights)

            self.dataMatrix_weighted = self.dataMatrix.T.dot(self.row_weights_diag).T

> **Recommenders**

In [12]:
# BaseRecommender.py
# ------------------------------------------------------------------


#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: Maurizio Ferrari Dacrema
"""

import numpy as np

# from utils.compute_similarity import check_matrix

class BaseRecommender(object):
    """Abstract BaseRecommender"""

    RECOMMENDER_NAME = "Recommender_Base_Class"

    def __init__(self, URM_train, verbose=True):

        super(BaseRecommender, self).__init__()

        self.URM_train = check_matrix(URM_train.copy(), 'csr', dtype=np.float32)
        self.URM_train.eliminate_zeros()

        self.n_users, self.n_items = self.URM_train.shape
        self.verbose = verbose

        self.filterTopPop = False
        self.filterTopPop_ItemsID = np.array([], dtype=np.int)

        self.items_to_ignore_flag = False
        self.items_to_ignore_ID = np.array([], dtype=np.int)

        self._cold_user_mask = np.ediff1d(self.URM_train.indptr) == 0

        if self._cold_user_mask.any():
            self._print("URM Detected {} ({:.2f} %) cold users.".format(
                self._cold_user_mask.sum(), self._cold_user_mask.sum()/self.n_users*100))


        self._cold_item_mask = np.ediff1d(self.URM_train.tocsc().indptr) == 0

        if self._cold_item_mask.any():
            self._print("URM Detected {} ({:.2f} %) cold items.".format(
                self._cold_item_mask.sum(), self._cold_item_mask.sum()/self.n_items*100))


    def _get_cold_user_mask(self):
        return self._cold_user_mask

    def _get_cold_item_mask(self):
        return self._cold_item_mask


    def _print(self, string):
        if self.verbose:
            print("{}: {}".format(self.RECOMMENDER_NAME, string))

    def fit(self):
        pass

    def get_URM_train(self):
        return self.URM_train.copy()

    def set_items_to_ignore(self, items_to_ignore):
        self.items_to_ignore_flag = True
        self.items_to_ignore_ID = np.array(items_to_ignore, dtype=np.int)

    def reset_items_to_ignore(self):
        self.items_to_ignore_flag = False
        self.items_to_ignore_ID = np.array([], dtype=np.int)


    #########################################################################################################
    ##########                                                                                     ##########
    ##########                     COMPUTE AND FILTER RECOMMENDATION LIST                          ##########
    ##########                                                                                     ##########
    #########################################################################################################


    def _remove_TopPop_on_scores(self, scores_batch):
        scores_batch[:, self.filterTopPop_ItemsID] = -np.inf
        return scores_batch


    def _remove_custom_items_on_scores(self, scores_batch):
        scores_batch[:, self.items_to_ignore_ID] = -np.inf
        return scores_batch


    def _remove_seen_on_scores(self, user_id, scores):

        assert self.URM_train.getformat() == "csr", "Recommender_Base_Class: URM_train is not CSR, this will cause errors in filtering seen items"

        seen = self.URM_train.indices[self.URM_train.indptr[user_id]:self.URM_train.indptr[user_id + 1]]

        scores[seen] = -np.inf
        return scores


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        """
        :param user_id_array:       array containing the user indices whose recommendations need to be computed
        :param items_to_compute:    array containing the items whose scores are to be computed.
                                        If None, all items are computed, otherwise discarded items will have as score -np.inf
        :return:                    array (len(user_id_array), n_items) with the score.
        """
        raise NotImplementedError("BaseRecommender: compute_item_score not assigned for current recommender, unable to compute prediction scores")


    def recommend(self, user_id_array, cutoff = None, remove_seen_flag=True, items_to_compute = None,
                  remove_top_pop_flag = False, remove_custom_items_flag = False, return_scores = False):

        # If is a scalar transform it in a 1-cell array
        if np.isscalar(user_id_array):
            user_id_array = np.atleast_1d(user_id_array)
            single_user = True
        else:
            single_user = False

        if cutoff is None:
            cutoff = self.URM_train.shape[1] - 1

        # Compute the scores using the model-specific function
        # Vectorize over all users in user_id_array
        scores_batch = self._compute_item_score(user_id_array, items_to_compute=items_to_compute)


        for user_index in range(len(user_id_array)):

            user_id = user_id_array[user_index]

            if remove_seen_flag:
                scores_batch[user_index,:] = self._remove_seen_on_scores(user_id, scores_batch[user_index, :])

            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index
            # relevant_items_partition = (-scores_user).argpartition(cutoff)[0:cutoff]
            # relevant_items_partition_sorting = np.argsort(-scores_user[relevant_items_partition])
            # ranking = relevant_items_partition[relevant_items_partition_sorting]
            #
            # ranking_list.append(ranking)


        if remove_top_pop_flag:
            scores_batch = self._remove_TopPop_on_scores(scores_batch)

        if remove_custom_items_flag:
            scores_batch = self._remove_custom_items_on_scores(scores_batch)

        # relevant_items_partition is block_size x cutoff
        relevant_items_partition = (-scores_batch).argpartition(cutoff, axis=1)[:,0:cutoff]

        # Get original value and sort it
        # [:, None] adds 1 dimension to the array, from (block_size,) to (block_size,1)
        # This is done to correctly get scores_batch value as [row, relevant_items_partition[row,:]]
        relevant_items_partition_original_value = scores_batch[np.arange(scores_batch.shape[0])[:, None], relevant_items_partition]
        relevant_items_partition_sorting = np.argsort(-relevant_items_partition_original_value, axis=1)
        ranking = relevant_items_partition[np.arange(relevant_items_partition.shape[0])[:, None], relevant_items_partition_sorting]

        ranking_list = [None] * ranking.shape[0]

        # Remove from the recommendation list any item that has a -inf score
        # Since -inf is a flag to indicate an item to remove
        for user_index in range(len(user_id_array)):
            user_recommendation_list = ranking[user_index]
            user_item_scores = scores_batch[user_index, user_recommendation_list]

            not_inf_scores_mask = np.logical_not(np.isinf(user_item_scores))

            user_recommendation_list = user_recommendation_list[not_inf_scores_mask]
            ranking_list[user_index] = user_recommendation_list.tolist()



        # Return single list for one user, instead of list of lists
        if single_user:
            ranking_list = ranking_list[0]


        if return_scores:
            return ranking_list, scores_batch

        else:
            return ranking_list


In [13]:
# BaseSimilarityMatrixRecommender.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 16/09/2017
@author: Maurizio Ferrari Dacrema
"""

# from recommenders.BaseRecommender import BaseRecommender
# from utils.DataIO import DataIO
import numpy as np


class BaseSimilarityMatrixRecommender(BaseRecommender):
    """
    This class refers to a BaseRecommender KNN which uses a similarity matrix, it provides two function to compute item's score
    bot for user-based and Item-based models as well as a function to save the W_matrix
    """

    def __init__(self, URM_train, verbose=True):
        super(BaseSimilarityMatrixRecommender, self).__init__(URM_train, verbose = verbose)

        self._URM_train_format_checked = False
        self._W_sparse_format_checked = False



    def _check_format(self):

        if not self._URM_train_format_checked:

            if self.URM_train.getformat() != "csr":
                self._print("PERFORMANCE ALERT compute_item_score: {} is not {}, this will significantly slow down the computation.".format("URM_train", "csr"))

            self._URM_train_format_checked = True

        if not self._W_sparse_format_checked:

            if self.W_sparse.getformat() != "csr":
                self._print("PERFORMANCE ALERT compute_item_score: {} is not {}, this will significantly slow down the computation.".format("W_sparse", "csr"))

            self._W_sparse_format_checked = True


    def save_model(self, folder_path, file_name = None):

        if file_name is None:
            file_name = self.RECOMMENDER_NAME

        self._print("Saving model in file '{}'".format(folder_path + file_name))

        data_dict_to_save = {"W_sparse": self.W_sparse}

        dataIO = DataIO(folder_path=folder_path)
        dataIO.save_data(file_name=file_name, data_dict_to_save = data_dict_to_save)

        self._print("Saving complete")



    #########################################################################################################
    ##########                                                                                     ##########
    ##########                               COMPUTE ITEM SCORES                                   ##########
    ##########                                                                                     ##########
    #########################################################################################################


class BaseItemSimilarityMatrixRecommender(BaseSimilarityMatrixRecommender):

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        """
        URM_train and W_sparse must have the same format, CSR
        :param user_id_array:
        :param items_to_compute:
        :return:
        """

        self._check_format()

        user_profile_array = self.URM_train[user_id_array]

        if items_to_compute is not None:
            item_scores = - np.ones((len(user_id_array), self.URM_train.shape[1]), dtype=np.float32)*np.inf
            item_scores_all = user_profile_array.dot(self.W_sparse).toarray()
            item_scores[:, items_to_compute] = item_scores_all[:, items_to_compute]
        else:
            item_scores = user_profile_array.dot(self.W_sparse).toarray()

        return item_scores


class BaseUserSimilarityMatrixRecommender(BaseSimilarityMatrixRecommender):

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        """
        URM_train and W_sparse must have the same format, CSR
        :param user_id_array:
        :param items_to_compute:
        :return:
        """

        self._check_format()

        user_weights_array = self.W_sparse[user_id_array]

        if items_to_compute is not None:
            item_scores = - np.ones((len(user_id_array), self.URM_train.shape[1]), dtype=np.float32)*np.inf
            item_scores_all = user_weights_array.dot(self.URM_train).toarray()
            item_scores[:, items_to_compute] = item_scores_all[:, items_to_compute]
        else:
            item_scores = user_weights_array.dot(self.URM_train).toarray()

        return item_scores

In [14]:
# UserKNNCFRecommender.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 23/10/17
@author: Maurizio Ferrari Dacrema
"""

# from utils.compute_similarity import check_matrix
# from recommenders.BaseSimilarityMatrixRecommender import BaseUserSimilarityMatrixRecommender

# from Base.IR_feature_weighting import okapi_BM_25, TF_IDF
import numpy as np

# from utils.compute_similarity import Compute_Similarity


class UserKNNCFRecommender(BaseUserSimilarityMatrixRecommender):
    """ UserKNN recommender"""

    RECOMMENDER_NAME = "UserKNNCFRecommender"

    FEATURE_WEIGHTING_VALUES = ["BM25", "TF-IDF", "none"]

    def __init__(self, URM_train, verbose = True):
        super(UserKNNCFRecommender, self).__init__(URM_train, verbose = verbose)



    def fit(self, topK=50, shrink=100, similarity='cosine', normalize=True, feature_weighting = "none", **similarity_args):

        self.topK = topK
        self.shrink = shrink

        if feature_weighting not in self.FEATURE_WEIGHTING_VALUES:
            raise ValueError("Value for 'feature_weighting' not recognized. Acceptable values are {}, provided was '{}'".format(self.FEATURE_WEIGHTING_VALUES, feature_weighting))


        # if feature_weighting == "BM25":
        #     self.URM_train = self.URM_train.astype(np.float32)
        #     self.URM_train = okapi_BM_25(self.URM_train.T).T
        #     self.URM_train = check_matrix(self.URM_train, 'csr')
        #
        # elif feature_weighting == "TF-IDF":
        #     self.URM_train = self.URM_train.astype(np.float32)
        #     self.URM_train = TF_IDF(self.URM_train.T).T
        #     self.URM_train = check_matrix(self.URM_train, 'csr')

        similarity = Compute_Similarity(self.URM_train.T, shrink=shrink, topK=topK, normalize=normalize, similarity = similarity, **similarity_args)

        self.W_sparse = similarity.compute_similarity()
        self.W_sparse = check_matrix(self.W_sparse, format='csr')

In [15]:
# ItemKNNCFRecommender.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 23/10/17
@author: Maurizio Ferrari Dacrema
"""

# from utils.compute_similarity import check_matrix
# from recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender

# from Base.IR_feature_weighting import okapi_BM_25, TF_IDF
import numpy as np

# from utils.compute_similarity import Compute_Similarity


class ItemKNNCFRecommender(BaseItemSimilarityMatrixRecommender):
    """ ItemKNN recommender"""

    RECOMMENDER_NAME = "ItemKNNCFRecommender"

    FEATURE_WEIGHTING_VALUES = ["BM25", "TF-IDF", "none"]

    def __init__(self, URM_train, verbose = True):
        super(ItemKNNCFRecommender, self).__init__(URM_train, verbose = verbose)


    def fit(self, topK=50, shrink=100, similarity='cosine', normalize=True, feature_weighting = "none", **similarity_args):

        self.topK = topK
        self.shrink = shrink

        if feature_weighting not in self.FEATURE_WEIGHTING_VALUES:
            raise ValueError("Value for 'feature_weighting' not recognized. Acceptable values are {}, provided was '{}'".format(self.FEATURE_WEIGHTING_VALUES, feature_weighting))


        # if feature_weighting == "BM25":
        #     self.URM_train = self.URM_train.astype(np.float32)
        #     self.URM_train = okapi_BM_25(self.URM_train.T).T
        #     self.URM_train = check_matrix(self.URM_train, 'csr')
        #
        # elif feature_weighting == "TF-IDF":
        #     self.URM_train = self.URM_train.astype(np.float32)
        #     self.URM_train = TF_IDF(self.URM_train.T).T
        #     self.URM_train = check_matrix(self.URM_train, 'csr')

        similarity = Compute_Similarity(self.URM_train, shrink=shrink, topK=topK, normalize=normalize, similarity = similarity, **similarity_args)


        self.W_sparse = similarity.compute_similarity()
        self.W_sparse = check_matrix(self.W_sparse, format='csr')

In [16]:
# ItemKNNCBFRecommender.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 23/10/17
@author: Maurizio Ferrari Dacrema
"""

# from utils.compute_similarity import check_matrix
# from recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender

# from Base.IR_feature_weighting import okapi_BM_25, TF_IDF
import numpy as np

# from utils.compute_similarity import Compute_Similarity


class ItemKNNCBFRecommender(BaseItemSimilarityMatrixRecommender):
    """ ItemKNN recommender"""

    RECOMMENDER_NAME = "ItemKNNCBFRecommender"

    FEATURE_WEIGHTING_VALUES = ["BM25", "TF-IDF", "none"]

    def __init__(self, URM_train, ICM_train, verbose=True):
        super(ItemKNNCBFRecommender, self).__init__(URM_train, verbose=verbose)

        self.ICM_train = ICM_train

    def fit(self, topK=50, shrink=100, similarity='cosine', normalize=True, feature_weighting="none",
            **similarity_args):

        self.topK = topK
        self.shrink = shrink

        if feature_weighting not in self.FEATURE_WEIGHTING_VALUES:
            raise ValueError(
                "Value for 'feature_weighting' not recognized. Acceptable values are {}, provided was '{}'".format(
                    self.FEATURE_WEIGHTING_VALUES, feature_weighting))

        # if feature_weighting == "BM25":
        #     self.ICM_train = self.ICM_train.astype(np.float32)
        #     self.ICM_train = okapi_BM_25(self.ICM_train)
        #
        # elif feature_weighting == "TF-IDF":
        #     self.ICM_train = self.ICM_train.astype(np.float32)
        #     self.ICM_train = TF_IDF(self.ICM_train)

        similarity = Compute_Similarity(self.ICM_train.T, shrink=shrink, topK=topK, normalize=normalize,
                                        similarity=similarity, **similarity_args)

        self.W_sparse = similarity.compute_similarity()
        self.W_sparse = check_matrix(self.W_sparse, format='csr')

In [17]:
# BaseMatrixFactorizationRecommender.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 16/09/2017
@author: Maurizio Ferrari Dacrema
"""

# from recommenders.BaseRecommender import BaseRecommender
# from KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

import numpy as np


class BaseMatrixFactorizationRecommender(BaseRecommender):
    """
    This class refers to a BaseRecommender KNN which uses matrix factorization,
    it provides functions to compute item's score as well as a function to save the W_matrix
    The prediction for cold users will always be -inf for ALL items
    """

    def __init__(self, URM_train, verbose=True):
        super(BaseMatrixFactorizationRecommender, self).__init__(URM_train, verbose=verbose)

        self.use_bias = False

    #########################################################################################################
    ##########                                                                                     ##########
    ##########                               COMPUTE ITEM SCORES                                   ##########
    ##########                                                                                     ##########
    #########################################################################################################


    def _compute_item_score(self, user_id_array, items_to_compute = None):
        """
        USER_factors is n_users x n_factors
        ITEM_factors is n_items x n_factors
        The prediction for cold users will always be -inf for ALL items
        :param user_id_array:
        :param items_to_compute:
        :return:
        """

        assert self.USER_factors.shape[1] == self.ITEM_factors.shape[1], \
            "{}: User and Item factors have inconsistent shape".format(self.RECOMMENDER_NAME)

        assert self.USER_factors.shape[0] > np.max(user_id_array),\
                "{}: Cold users not allowed. Users in trained model are {}, requested prediction for users up to {}".format(
                self.RECOMMENDER_NAME, self.USER_factors.shape[0], np.max(user_id_array))

        if items_to_compute is not None:
            item_scores = - np.ones((len(user_id_array), self.ITEM_factors.shape[0]), dtype=np.float32)*np.inf
            item_scores[:, items_to_compute] = np.dot(self.USER_factors[user_id_array], self.ITEM_factors[items_to_compute,:].T)

        else:
            item_scores = np.dot(self.USER_factors[user_id_array], self.ITEM_factors.T)


        # No need to select only the specific negative items or warm users because the -inf score will not change
        if self.use_bias:
            item_scores += self.ITEM_bias + self.GLOBAL_bias
            item_scores = (item_scores.T + self.USER_bias[user_id_array]).T

        return item_scores


In [18]:
# PureSVDRecommender.py
# ------------------------------------------------------------------

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 14/06/18
@author: Maurizio Ferrari Dacrema
"""

# from recommenders.BaseMatrixFactorizationRecommender import BaseMatrixFactorizationRecommender

from sklearn.utils.extmath import randomized_svd
import scipy.sparse as sps


class PureSVDRecommender(BaseMatrixFactorizationRecommender):
    """ PureSVDRecommender"""

    RECOMMENDER_NAME = "PureSVDRecommender"

    def __init__(self, URM_train, verbose = True):
        super(PureSVDRecommender, self).__init__(URM_train, verbose = verbose)


    def fit(self, num_factors=100, random_seed = None):

        self._print("Computing SVD decomposition...")

        U, Sigma, VT = randomized_svd(self.URM_train,
                                      n_components=num_factors,
                                      #n_iter=5,
                                      random_state = random_seed)

        s_Vt = sps.diags(Sigma)*VT

        self.USER_factors = U
        self.ITEM_factors = s_Vt.T

        self._print("Computing SVD decomposition... Done!")

In [19]:
# main.py
# ------------------------------------------------------------------

import os, traceback
import numpy as np

URM_all = build_URM()
ICM_all = build_ICM()
# data_manager.get_statistics_URM(URM)


######################################################################
##########                                                  ##########
##########      TRAINING, EVALUATION AND PREDICTIONS        ##########
##########                                                  ##########
######################################################################


# URM train/validation/test splitting
# -----------------------------------

# from Data_manager.Movielens1M.Movielens1MReader import Movielens1MReader
# from Data_manager.DataSplitter_k_fold_stratified import DataSplitter_Warm_k_fold

# dataset_object = Movielens1MReader()
# dataSplitter = DataSplitter_Warm_k_fold(dataset_object)
# dataSplitter.load_data()
# URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

URM_train, URM_test = split_train_validation_random_holdout(URM_all, train_split=0.8)
URM_train, URM_validation = split_train_validation_random_holdout(URM_train, train_split=0.9)


# Tuning parameters
# -----------------

metric_to_optimize = "MAP"

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10, 15])
evaluator_validation_earlystopping = EvaluatorHoldout(URM_train, cutoff_list=[10], exclude_seen = False) # None 
output_folder_path = "result_experiments/"

n_cases = 8 # 2
n_random_starts = 5  # 1

save_model = "no"
allow_weighting = False
similarity_type_list = ["cosine"]

ICM_name = "ICM_all"

# Collaborative recommenders
collaborative_algorithm_list = [
    # Random,
    # TopPop,
    # P3alphaRecommender,
    # RP3betaRecommender,
    ItemKNNCFRecommender,
    UserKNNCFRecommender,
#     MatrixFactorization_BPR_Cython,
    # MatrixFactorization_FunkSVD_Cython,
    PureSVDRecommender,
    # SLIM_BPR_Cython,
    # SLIMElasticNetRecommender
]

# Content-based recommenders
content_algorithm_list = [
    ItemKNNCBFRecommender
]

recommender_list = [
    # Random,
    # TopPop,
    # P3alphaRecommender,
    # RP3betaRecommender,
    ItemKNNCFRecommender,
    UserKNNCFRecommender,
#     MatrixFactorization_BPR_Cython,
    # MatrixFactorization_FunkSVD_Cython,
    PureSVDRecommender,
    # SLIM_BPR_Cython,
    # SLIMElasticNetRecommender,
    ItemKNNCBFRecommender
]


# from Utils.PoolWithSubprocess import PoolWithSubprocess
# import multiprocessing
#
# pool = PoolWithSubprocess(processes=int(multiprocessing.cpu_count()), maxtasksperchild=1)
# resultList = pool.map(runParameterSearch_Collaborative_partial, collaborative_algorithm_list)
# pool.close()
# pool.join()


print('\nRecommender Systems: ')
for i, recomm_type in enumerate(recommender_list, start=1):
    print('{}. {}'.format(i, recomm_type.RECOMMENDER_NAME))

while True:
    try:
        selected = int(input('\nSelect a recommender system: '.format(i)))
        recommender_class = recommender_list[selected - 1]
        print('\n ... {} ... '.format(recommender_class.RECOMMENDER_NAME))

        output_file_name_root = "{}_metadata.zip".format(recommender_class.RECOMMENDER_NAME)


        # Hyperparameters tuning
        apply_hyperparams_tuning = False
        
        if apply_hyperparams_tuning :
            
            if recommender_class in collaborative_algorithm_list:

                try:
                    runParameterSearch_Collaborative(recommender_class=recommender_class,
                                                     URM_train=URM_train,
                                                     metric_to_optimize=metric_to_optimize,
                                                     evaluator_validation=evaluator_validation,
                                                     evaluator_test=evaluator_test,
                                                     evaluator_validation_earlystopping=evaluator_validation_earlystopping,
                                                     output_folder_path=output_folder_path,
                                                     n_cases=n_cases,
                                                     n_random_starts=n_random_starts,
                                                     save_model=save_model,
                                                     allow_weighting=allow_weighting,
                                                     similarity_type_list=similarity_type_list)

                    if recommender_class in [ItemKNNCFRecommender, UserKNNCFRecommender]:
                        similarity_type = similarity_type_list[0]  # KNN Recommenders on similarity_type
                        output_file_name_root = "{}_{}_metadata.zip".format(recommender_class.RECOMMENDER_NAME,
                                                                            similarity_type)

                except Exception as e:
                    print("On recommender {} Exception {}".format(recommender_class, str(e)))
                    traceback.print_exc()


            if recommender_class in content_algorithm_list:

                try:
                    runParameterSearch_Content(recommender_class=recommender_class,
                                               URM_train=URM_train,
                                               ICM_object=ICM_all,
                                               ICM_name=ICM_name,
                                               n_cases=n_cases,
                                               n_random_starts=n_random_starts,
                                               save_model=save_model,
                                               evaluator_validation=evaluator_validation,
                                               evaluator_test=evaluator_test,
                                               metric_to_optimize=metric_to_optimize,
                                               output_folder_path=output_folder_path,
                                               allow_weighting=allow_weighting,
                                               similarity_type_list=similarity_type_list)

                    similarity_type = similarity_type_list[0]  # KNN Recommenders on similarity_type
                    output_file_name_root = "{}_{}_{}_metadata.zip".format(recommender_class.RECOMMENDER_NAME,
                                                                           ICM_name, similarity_type)

                except Exception as e:
                    print("On recommender {} Exception {}".format(recommender_class, str(e)))
                    traceback.print_exc()

                # Load best_parameters for training
                data_loader = DataIO(folder_path=output_folder_path)
                search_metadata = data_loader.load_data(output_file_name_root)
                best_parameters = search_metadata["hyperparameters_best"]  # dictionary with all the fit parameters
                print("best_parameters {}".format(best_parameters))
        
        else:
           
            best_parameters = {'topK': 5, 'shrink': 1000, 'similarity': 'cosine', 'normalize': True}
            
            
        # Fit the recommender with the parameters we just learned
        if recommender_class in content_algorithm_list:
            # todo: ICM_all or ICM_train?
            recommender = recommender_class(URM_train, ICM_all)
        else:
            recommender = recommender_class(URM_train)
        
        recommender.fit(**best_parameters)

        # Evaluate model
        evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
        result_dict, _ = evaluator_test.evaluateRecommender(recommender)

        print("{} result_dict MAP {}".format(recommender_class.RECOMMENDER_NAME, result_dict[10]["MAP"]))

        # Make predictions
        predictions = input('\nCompute and save top10 predictions?: y - Yes  n - No\n')

        if predictions == 'y':
            # Train the model on the whole dataset using tuned params
            if recommender_class in content_algorithm_list:
                # todo: ICM_all or ICM_train?
                recommender = recommender_class(URM_all, ICM_all)
            else:
                recommender = recommender_class(URM_all)

            recommender.fit(**best_parameters)

            top_10_items = {}
            target_user_id_list = get_target_users()

            for user_id in target_user_id_list:
                item_list = ''
                for item in range(10):  # recommended_items
                    item_list = recommender.recommend(user_id, cutoff=10)
                    item_list = np.array(item_list)  # list to np.array
                    top_10_items[user_id] = item_list  # .strip() # remove trailing space

            # save predictions on csv file
            create_csv(top_10_items, recommender_class.RECOMMENDER_NAME)

        break

    except (ValueError, IndexError):
        print('Error. Please enter number between 1 and {}'.format(i))


IndentationError: expected an indented block (<ipython-input-19-4be8d475dfdf>, line 117)